In [1]:
# import import_ipynb
# import oss2
# from setting import SETTINGS
# import pickle
# import pandas as pd
# import os
# import io
# from dateutil import parser
# import datetime
# import pandas as pd
# from fastparquet import write
# from pathlib import  Path

# AccessKeyId = SETTINGS["oss.accesskey"]
# AccessKeySecret = SETTINGS["oss.secret"]
# BucketName = SETTINGS["oss.bucketname"]
# Endpoint = SETTINGS["oss.endpoint"]


# class newBytes(io.BytesIO):
#     def close(self):
#         pass


# class OssClient(object):
#     __instance = None
#     __first_init = False

#     # 单例模式
#     def __new__(cls, *args, **kwargs):
#         if not cls.__instance:
#             cls.__instance = super().__new__(cls)
#         return cls.__instance

#     def __init__(self):
#         cls = self.__class__
#         if not cls.__first_init:
#             self.auth = oss2.Auth(AccessKeyId, AccessKeySecret)
#             self.bucket = oss2.Bucket(self.auth, Endpoint, BucketName)
#             cls.__first_init = True


#     def upload_file_from_fileobj(self, object_name, local_file_path):
#         """
#             upload_file_from_fileobj方法：上传文件对象到oss存储空间, 该方法可用于我们从上游服务接收了图片参数，然后以二进制形式读文件，上传到oss存储空间指定位置（abc/efg/00），
#         当然也可以将本地文件上传到oss我们的bucket. 其中fileobj不止可以是文件对象，也可以是本地文件路径。 put_object方法底层仍是RESTful API的调用，可以指定headers，规定Content-Type等内容
#         """
#         # 判断bucket中文件是否存在，也可以不判断，会上传更新
#         #exist = self.bucket.object_exists(object_name) #<yourObjectName>
#         #if exist:
#         #    return True
#         with open(local_file_path, 'rb') as fileobj:
#             result = self.bucket.put_object(object_name, fileobj) #<yourObjectName>
#         if result.status == 200:
#             return True
#         else:
#             return False

#     def upload_pickle_data(self,df, target_path, *args, report_date=None):
#         if isinstance(report_date, datetime.date):
#             d = report_date.strftime("%Y-%m-%d")
#         if isinstance(report_date, str):
#             d = parser.parse(report_date).strftime("%Y-%m-%d")
#         if args:
#             d = [arg for arg in args][0]
#         pickle_buffer = io.BytesIO()
#         pickle.dump(df, pickle_buffer)
#         target_file_key = os.path.join(target_path, '{}.pkl'.format(d)).replace("\\","/")
#         result = self.bucket.put_object(target_file_key, pickle_buffer.getvalue())
#         if result.status == 200:
#             return True
#         else:
#             return False


#     def list_files(self,prefix = None):
#         res = []
#         for object_info in oss2.ObjectIterator(self.bucket,prefix):
#             print(object_info.key)
#             res.append(object_info.key)
#         return res

#     def upload_parquet_data(self,df, target_path, *args, report_date=None):
#         if isinstance(report_date, datetime.date):
#             d = report_date.strftime("%Y-%m-%d")
#         if isinstance(report_date, str):
#             d = parser.parse(report_date).strftime("%Y-%m-%d")
#         if args:
#             d = [arg for arg in args][0]
#         target_file_key = os.path.join(target_path, '{}.parquet'.format(d)).replace("\\","/")
#         mem_buffer = newBytes()
#         df.to_parquet('noname', engine='fastparquet', open_with=lambda x, y: mem_buffer)
#         result = self.bucket.put_object(target_file_key, mem_buffer.getvalue())
#         #f = Path(os.getcwd())/'tmp.parquet'
#         #write(f, df)
#         #with open(f, 'rb') as fileobj:
#         #    result = self.bucket.put_object(target_file_key, fileobj)
#         #os.remove(f)
#         if result.status == 200:
#             return True
#         else:
#             return False

#     def save_data_to_pickle(self,df, file_dir_path, *args,report_date=None):
#         if isinstance(report_date, datetime.date):
#             d = report_date.strftime("%Y-%m-%d")
#         if isinstance(report_date, str):
#             d = parser.parse(report_date).strftime("%Y-%m-%d")
#         if args:
#             d = [arg for arg in args][0]
#         target_file_key = os.path.join(file_dir_path, '{}.pkl'.format(d))
#         with open(target_file_key, 'wb') as f:
#             pickle.dump(df, f)

#     def save_data_to_parquet(self,df, file_dir_path, *args,report_date=None):
#         if isinstance(report_date, datetime.date):
#             d = report_date.strftime("%Y-%m-%d")
#         if isinstance(report_date, str):
#             d = parser.parse(report_date).strftime("%Y-%m-%d")
#         if args:
#             d = [arg for arg in args][0]
#         target_file_key = os.path.join(file_dir_path, f'{d}.parquet')
#         df.to_parquet(target_file_key)


#     def read_oss_pickle_file(self,object_name):
#         """
#             download_file_to_fileobj：下载文件到文件流对象。由于get_object接口返回的是一个stream流，需要执行read()后才能计算出返回Object数据的CRC checksum，因此需要在调用该接口后做CRC校验。
#         """
#         object_stream = self.bucket.get_object(object_name) #<yourObjectName>
#         result = object_stream.read()
#         if object_stream.client_crc != object_stream.server_crc:
#             print("The CRC checksum between client and server is inconsistent!")
#             result = None
#         return pickle.loads(result)

#     def read_oss_parquet_file(self,object_name):
#         """
#             download_file_to_fileobj：下载文件到文件流对象。由于get_object接口返回的是一个stream流，需要执行read()后才能计算出返回Object数据的CRC checksum，因此需要在调用该接口后做CRC校验。
#         """
#         object_stream = self.bucket.get_object(object_name) #<yourObjectName>
#         result = object_stream.read()
#         i = io.BytesIO(result)
#         if object_stream.client_crc != object_stream.server_crc:
#             print("The CRC checksum between client and server is inconsistent!")
#             result = None
#         return pd.read_parquet(i)


#     def download_file_to_loaclfilepath(self, object_name, local_file_path):
#         """
#             download_file_to_loaclfilepath：下载文件到本地路径。get_object和get_object_to_file的区别是前者是获取文件流实例，可用于代码处理和远程调用参赛。后者是存储到本地路径，返回的是一个http状态的json结果
#         """
#         result = self.bucket.get_object_to_file(object_name, local_file_path) # ('<yourObjectName>', '<yourLocalFile>')
#         if result.status == 200:
#             return True
#         else:
#             return False

#     def generate_temporary_download_url(self,object_name):
#         """
#             generate_temporary_download_url: 生成加签的临时URL以供授信用户下载。一般在实际业务中，我们是提供给调用方一个临时下载链接，来让其获取文件数据，而不是直接使用以上暴露AccessKeyId和AccessKeySecret的方法。
#             因此一般我们会存储某条数据oss的路径（<yourObjectName>）与调用方某个唯一标识的对应关系（如手机号身份证号），在调用方请求时，通过该标识获取其数据的oss文件路径（<yourObjectName>），
#             然后制定过期时间，为其生成临时下载链接
#             http://bucketname.oss-ap-south-1.aliyuncs.com/abc/efg/0?OSSAccessKeyId=LTA************oN9&Expires=1604638842&Signature=tPgvWz*************Uk%3D
#         """
#         res_temporary_url = self.bucket.sign_url('GET', object_name, 60, slash_safe=True)
#         return res_temporary_url


# oss_client = OssClient()

importing Jupyter notebook from setting.ipynb


In [2]:
import import_ipynb
import oss2
from setting import SETTINGS
import pickle
import pandas as pd
import os
import io
from dateutil import parser
import datetime
import pandas as pd
from fastparquet import write
from pathlib import  Path

AccessKeyId = SETTINGS["oss.accesskey"]
AccessKeySecret = SETTINGS["oss.secret"]
BucketName = SETTINGS["oss.bucketname"]
Endpoint = SETTINGS["oss.endpoint"]


class newBytes(io.BytesIO):
    def close(self):
        pass


class OssClient(object):
    __instance = None
    __first_init = False

    # 单例模式
    def __new__(cls, *args, **kwargs):
        if not cls.__instance:
            cls.__instance = super().__new__(cls)
        return cls.__instance

    def __init__(self):
        cls = self.__class__
        if not cls.__first_init:
            self.auth = oss2.Auth(AccessKeyId, AccessKeySecret)
            self.bucket = oss2.Bucket(self.auth, Endpoint, BucketName)
            cls.__first_init = True


    def upload_file_from_fileobj(self, object_name, local_file_path):
        """
            upload_file_from_fileobj方法：上传文件对象到oss存储空间, 该方法可用于我们从上游服务接收了图片参数，然后以二进制形式读文件，上传到oss存储空间指定位置（abc/efg/00），
        当然也可以将本地文件上传到oss我们的bucket. 其中fileobj不止可以是文件对象，也可以是本地文件路径。 put_object方法底层仍是RESTful API的调用，可以指定headers，规定Content-Type等内容
        """
        # 判断bucket中文件是否存在，也可以不判断，会上传更新
        #exist = self.bucket.object_exists(object_name) #<yourObjectName>
        #if exist:
        #    return True
        with open(local_file_path, 'rb') as fileobj:
            result = self.bucket.put_object(object_name, fileobj) #<yourObjectName>
        if result.status == 200:
            return True
        else:
            return False

    def upload_pickle_data(self,df, target_path, *args, report_date=None):
        if isinstance(report_date, datetime.date):
            d = report_date.strftime("%Y-%m-%d")
        if isinstance(report_date, str):
            d = parser.parse(report_date).strftime("%Y-%m-%d")
        if args:
            d = [arg for arg in args][0]
        pickle_buffer = io.BytesIO()
        pickle.dump(df, pickle_buffer)
        target_file_key = os.path.join(target_path, '{}.pkl'.format(d)).replace("\\","/")
        result = self.bucket.put_object(target_file_key, pickle_buffer.getvalue())
        if result.status == 200:
            return True
        else:
            return False


    def list_files(self,prefix = None):
        res = []
        for object_info in oss2.ObjectIterator(self.bucket,prefix):
            print(object_info.key)
            res.append(object_info.key)
        return res

    def upload_parquet_data(self,df, target_path, *args, report_date=None):
        if isinstance(report_date, datetime.date):
            d = report_date.strftime("%Y-%m-%d")
        if isinstance(report_date, str):
            d = parser.parse(report_date).strftime("%Y-%m-%d")
        if args:
            d = [arg for arg in args][0]
        target_file_key = os.path.join(target_path, '{}.parquet'.format(d)).replace("\\","/")
        mem_buffer = newBytes()
        df.to_parquet('noname', engine='fastparquet', open_with=lambda x, y: mem_buffer)
        result = self.bucket.put_object(target_file_key, mem_buffer.getvalue())
        #f = Path(os.getcwd())/'tmp.parquet'
        #write(f, df)
        #with open(f, 'rb') as fileobj:
        #    result = self.bucket.put_object(target_file_key, fileobj)
        #os.remove(f)
        if result.status == 200:
            return True
        else:
            return False

    def save_data_to_pickle(self,df, file_dir_path, *args,report_date=None):
        if isinstance(report_date, datetime.date):
            d = report_date.strftime("%Y-%m-%d")
        if isinstance(report_date, str):
            d = parser.parse(report_date).strftime("%Y-%m-%d")
        if args:
            d = [arg for arg in args][0]
        target_file_key = os.path.join(file_dir_path, '{}.pkl'.format(d))
        with open(target_file_key, 'wb') as f:
            pickle.dump(df, f)

    def save_data_to_parquet(self,df, file_dir_path, *args,report_date=None):
        if isinstance(report_date, datetime.date):
            d = report_date.strftime("%Y-%m-%d")
        if isinstance(report_date, str):
            d = parser.parse(report_date).strftime("%Y-%m-%d")
        if args:
            d = [arg for arg in args][0]
        target_file_key = os.path.join(file_dir_path, f'{d}.parquet')
        df.to_parquet(target_file_key)


    def read_oss_pickle_file(self,object_name):
        """
            download_file_to_fileobj：下载文件到文件流对象。由于get_object接口返回的是一个stream流，需要执行read()后才能计算出返回Object数据的CRC checksum，因此需要在调用该接口后做CRC校验。
        """
        object_stream = self.bucket.get_object(object_name) #<yourObjectName>
        result = object_stream.read()
        if object_stream.client_crc != object_stream.server_crc:
            print("The CRC checksum between client and server is inconsistent!")
            result = None
        return pickle.loads(result)

    def read_oss_parquet_file(self,object_name):
        """
            download_file_to_fileobj：下载文件到文件流对象。由于get_object接口返回的是一个stream流，需要执行read()后才能计算出返回Object数据的CRC checksum，因此需要在调用该接口后做CRC校验。
        """
        object_stream = self.bucket.get_object(object_name) #<yourObjectName>
        result = object_stream.read()
        i = io.BytesIO(result)
        if object_stream.client_crc != object_stream.server_crc:
            print("The CRC checksum between client and server is inconsistent!")
            result = None
        return pd.read_parquet(i)


    def download_file_to_loaclfilepath(self, object_name, local_file_path):
        """
            download_file_to_loaclfilepath：下载文件到本地路径。get_object和get_object_to_file的区别是前者是获取文件流实例，可用于代码处理和远程调用参赛。后者是存储到本地路径，返回的是一个http状态的json结果
        """
        result = self.bucket.get_object_to_file(object_name, local_file_path) # ('<yourObjectName>', '<yourLocalFile>')
        if result.status == 200:
            return True
        else:
            return False

    def generate_temporary_download_url(self,object_name):
        """
            generate_temporary_download_url: 生成加签的临时URL以供授信用户下载。一般在实际业务中，我们是提供给调用方一个临时下载链接，来让其获取文件数据，而不是直接使用以上暴露AccessKeyId和AccessKeySecret的方法。
            因此一般我们会存储某条数据oss的路径（<yourObjectName>）与调用方某个唯一标识的对应关系（如手机号身份证号），在调用方请求时，通过该标识获取其数据的oss文件路径（<yourObjectName>），
            然后制定过期时间，为其生成临时下载链接
            http://bucketname.oss-ap-south-1.aliyuncs.com/abc/efg/0?OSSAccessKeyId=LTA************oN9&Expires=1604638842&Signature=tPgvWz*************Uk%3D
        """
        res_temporary_url = self.bucket.sign_url('GET', object_name, 60, slash_safe=True)
        return res_temporary_url


oss_client = OssClient()

importing Jupyter notebook from setting.ipynb


In [2]:
import import_ipynb
from oss_handler import OssClient
import sys, os
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from matplotlib import pyplot as plt
from pylab import mpl
import seaborn as sns
from sklearn.linear_model import LinearRegression
import datetime
from datetime import datetime, timedelta
from oss2.exceptions import NoSuchKey
import warnings
warnings.filterwarnings('ignore')
oss_client = OssClient()

importing Jupyter notebook from oss_handler.ipynb
LTAI5tN7V51xtxUmaKxmRsWo rmqsMszRtvurcckVMD9M14riQYshi6 2nd-data oss-cn-shenzhen.aliyuncs.com


# 说明

## Raw -- 已经完成的代码，但是目前用不到。
## Markdown -- 需要改进的代码，不能使用。

# 数据调用

In [7]:
# idx000001 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000001.XSHG_hist_index.pkl') #上证指数
# idx000016 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000016.XSHG_hist_index.pkl') #上证 50
# idx000300 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000300.XSHG_hist_index.pkl') #沪深 300
idx000852 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000852.XSHG_hist_index.pkl') #中证 1000
idx000905 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000905.XSHG_hist_index.pkl') #中证 500
idx000985 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/000985.XSHG_hist_index.pkl') #中证全指
# idx399001 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/399001.XSHG_hist_index.pkl') #深证成指
# idx399006 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/399006.XSHG_hist_index.pkl') #创业板指
# idx399303 = oss_client.read_oss_pickle_file('ad_hoc_prod/index_hist/399303.XSHG_hist_index.pkl') #国证 2000

In [8]:
adj_close = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/adj_close.pk')
adj_factor = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/adj_factor.pk')
adj_high = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/adj_high.pk')
adj_low = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/adj_low.pk')
adj_open = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/adj_open.pk')

circulation_a = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/circulation_a.pk') #流通股本
total_a = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/total_a.pk') #总股本
circulation_market_value = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/circulation_market_value.pk') #流通市值

volume = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/volume.pk') #成交量
num_trades = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/num_trades.pk') #交易次数
total_turnover = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/total_turnover.pk') #成交额
turnover_rate = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/turnover_rate.pk') #换手率

close = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/close.pk') 
open = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/open.pk') 
high = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/high.pk') 
low = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/low.pk') 

limit_down = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/limit_down.pk') #跌停
limit_up = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/limit_up.pk') #涨停

halt_status = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/halt_status.pk') #停牌 
st_status = oss_client.read_oss_pickle_file('ad_hoc_prod/fields_full/st_status.pk') #st

In [9]:
BS_pit = oss_client.read_oss_pickle_file('ad_hoc_prod/pit_fundemental/BS_pit.pkl')
CF_pit = oss_client.read_oss_pickle_file('ad_hoc_prod/pit_fundemental/CF_pit.pkl')
IS_pit = oss_client.read_oss_pickle_file('ad_hoc_prod/pit_fundemental/IS_pit.pkl')
all_pit = oss_client.read_oss_pickle_file('ad_hoc_prod/pit_fundemental/all_pit_fund_new_api.pkl')

In [10]:
##正常交易非st，halt的票打1.其他为np.nan
st_status = st_status.replace(True,np.nan).replace(False,True)
halt_status = halt_status.replace(True,np.nan).replace(False,True)

adj_return =  adj_close/adj_close.shift(1) - 1
adj_return = adj_return.replace(np.inf,np.nan).replace(-np.inf,np.nan)

##调仓周期
o1o2 = adj_open.shift(-2)/adj_open.shift(-1) - 1 #1日调仓
# o1o2 = adj_open.shift(-5) / adj_open.shift(-1) - 1 #5日调仓
# o1o2 = adj_open.shift(-10) / adj_open.shift(-1) - 1 #10日调仓
# o1o2 = adj_open.shift(-20) / adj_open.shift(-1) - 1 #20日调仓

mc = total_a * adj_close

In [11]:
csi500 = idx000905[['open','close','low']].set_index(idx000905['date'])
idx_500_ret = (csi500['close']/csi500['close'].shift(1) - 1)

mclose = csi500['close']
mopen = csi500['open']
mlow = csi500['low']

open['label'] = 1

# 中性化函数

In [12]:
logcmv = np.log(circulation_market_value)
logcmv = logcmv.loc['2017':]
logcmv = logcmv.stack().reset_index()
logcmv.columns = ['date','ticker','logcmv']
logcmv.to_pickle('D:/redata/risk_factors/logcmv.pkl')
logcmv.tail(1)

,date,ticker,logcmv
8272228,2025-02-17,689009,21.842942


In [13]:
retsum120 = adj_return.rolling(window=120).sum()
retsum120 = retsum120.loc['2017':]
retsum120 = retsum120.stack().reset_index()
retsum120.columns = ['date','ticker','retsum120']
retsum120.to_pickle('D:/redata/risk_factors/retsum120.pkl')
retsum120.tail(1)

,date,ticker,retsum120
8013866,2025-02-17,689009,0.249438


# 行业打标多天

ind = pd.read_pickle('D:/redata/risk_factors/ind_label_multiple_dates.pkl')
ind

starttime = '2025-02-10'
endtime = '2025-02-14'

ind_label_proc = pd.DataFrame(columns= ['date','ticker','first_industry_code'])

# 定义一个函数来生成工作日日期范围
def date_range(start, end):
    start_date = datetime.strptime(start, '%Y-%m-%d')
    end_date = datetime.strptime(end, '%Y-%m-%d')
    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() < 5:  # 0-4代表周一到周五
            yield current_date.strftime('%Y-%m-%d')
        current_date += timedelta(days=1)

for date in date_range(starttime, endtime):
    ind_label_raw = oss_client.read_oss_pickle_file(f'ad_hoc_prod/sector/{date}.pkl')
    temp_df = pd.DataFrame(columns=['date', 'ticker', 'first_industry_code'])
    temp_df['ticker'] = ind_label_raw.index
    temp_df['ticker'] = temp_df['ticker'].str.extract(r'(\d+)')
    temp_df['date'] = ind_label_raw['date'].values
    temp_df['first_industry_code'] = ind_label_raw['first_industry_code'].values
    
    # 将临时 DataFrame 追加到最终的 DataFrame
    # ind_label_proc = ind_label_proc.append(temp_df, ignore_index=True)
    ind_label_proc = pd.concat([ind_label_proc, temp_df], ignore_index=True)

temp_df = ind_label_proc

df1 = pd.DataFrame(columns= ['date','ticker','10','11','12','20','21','22','23','24','25','26','27','28','30','31','32','33','34','35','36','37','40','41','42','43','50','60','61','62','63','70','999'])
df1['date'] = temp_df['date']
df1['ticker'] = temp_df['ticker']

for i, code in enumerate(temp_df['first_industry_code']):
    if code in df1.columns:
        df1.at[i, code] = 1
    else:
        df1.at[i, code] = 0
        
df1 = df1.fillna(0)
df1

ind_label = pd.concat([ind, df1], ignore_index=True)
ind_label['date'] = pd.to_datetime(ind_label['date'])
ind_label = ind_label.reset_index(drop=True)
ind_label = pd.DataFrame(ind_label)
ind_label.to_pickle('D:/redata/risk_factors/ind_label_multiple_dates.pkl')
ind_label

# 单天打标

In [19]:
# ind = pd.read_pickle('D:/redata/risk_factors/ind_label_multiple_dates.pkl')
ind = pd.read_pickle('D:/redata/risk_factors/ind_label.pkl')
ind.tail(1)

,date,ticker,10,11,12,20,21,22,23,24,...,41,42,43,50,60,61,62,63,70,999
9041329,2025-02-21,689009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
date_data = '2025-02-24'

ind_label_raw = oss_client.read_oss_pickle_file(f'ad_hoc_prod/sector/{date_data}.pkl')

ind_label_proc = pd.DataFrame(columns= ['date','ticker','first_industry_code'])

temp_df = pd.DataFrame(columns=['date', 'ticker', 'first_industry_code'])
temp_df['ticker'] = ind_label_raw.index
temp_df['ticker'] = temp_df['ticker'].str.extract(r'(\d+)')
temp_df['date'] = ind_label_raw['date'].values
temp_df['first_industry_code'] = ind_label_raw['first_industry_code'].values
ind_label_proc = pd.concat([ind_label_proc, temp_df], ignore_index=True)

temp_df = ind_label_proc

df1 = pd.DataFrame(columns= ['date','ticker','10','11','12','20','21','22','23','24','25','26','27','28','30','31','32','33','34','35','36','37','40','41','42','43','50','60','61','62','63','70','999'])
df1['date'] = temp_df['date']
df1['ticker'] = temp_df['ticker']

for i, code in enumerate(temp_df['first_industry_code']):
    if code in df1.columns:
        df1.at[i, code] = 1
    else:
        df1.at[i, code] = 0
        
df1 = df1.fillna(0)

C:\Users\sensen.t\AppData\Local\Temp\ipykernel_13820\470789045.py:26: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1 = df1.fillna(0)


In [21]:
ind_label = pd.concat([ind, df1], ignore_index=True)
ind_label['date'] = pd.to_datetime(ind_label['date'])
ind_label = ind_label.reset_index(drop=True)
ind_label = pd.DataFrame(ind_label)
ind_label.to_pickle('D:/redata/risk_factors/ind_label.pkl')
ind_label

,date,ticker,10,11,12,20,21,22,23,24,...,41,42,43,50,60,61,62,63,70,999
0,2016-01-04,000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-04,000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016-01-04,000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-01-04,000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016-01-04,000006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9046463,2025-02-24,688799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9046464,2025-02-24,688800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9046465,2025-02-24,688819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9046466,2025-02-24,688981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


# 动量市值中性化

In [105]:
##因子中性化 需要的数据
risk1 = pd.read_pickle('D:/redata/risk_factors/logcmv.pkl')
risk2 = pd.read_pickle('D:/redata/risk_factors/retsum120.pkl')
ind = pd.read_pickle('D:/redata/risk_factors/ind_label.pkl')
# ind = pd.read_pickle('D:/redata/risk_factor/ind_label.pk')
risk = risk1.merge(risk2,on=['date','ticker'],how='left')
risk = risk.merge(ind,on=['date','ticker'],how='left')
risk = risk[(risk['date']>'2017-01-01')]
risk = risk.drop_duplicates(keep='last')

In [106]:
##行业中性 2023.8.3 更新
def neturalize_ind(factor,risk=risk,risk_cols=['logcmv','retsum120']):
    XY = pd.merge(risk, factor, on=['date', 'ticker'], how='right')
    #print(XY.head(3))
    XY = XY.dropna()
    tradeday = XY.date.unique()
    daily_resid = []
    for date in tqdm(tradeday[:]):
        xy = XY[XY['date'] == date]
        #只保留有因子值的数据
        xy = xy.dropna()

        #数据标准化
        xy[risk_cols] = (xy[risk_cols] - xy[risk_cols].mean())/xy[risk_cols].std()                
        xy['factor'] = (xy['factor']-xy['factor'].mean())/xy['factor'].std() 

        #选出x y
        x = xy.iloc[:,2:-1]
        y = xy['factor']
        #回归
        multi_linear = LinearRegression(fit_intercept=False)  #无截距项
        multi_linear.fit(x, y)
        beta = multi_linear.coef_
        
        #取残差
        xy['beta'] = np.dot(x, beta)
        xy['resid'] = xy['factor'] - xy['beta']
        daily_resid.append(xy)
    data = pd.concat(daily_resid)
    factor = data[['date', 'ticker', 'factor', 'resid']]
    return factor

In [107]:
def factor_neutralize(df,factor_type,if_rank=True,risk=None,risk_cols=None): #factor_type=hist原值；ind行业及其他中性；单个风险因子文件名； if_rank：中性化之前时候对因子做rank处理
    #中性之前是否对因子做rank处理
    if if_rank == True:
        df = df.rank(axis=1, ascending=True)  # 原值越大rank值越大   
    #因子原值
    if factor_type == 'hist':
        factor_rank = df.copy()
    #行业市值中性
    elif factor_type == 'ind':  ###行业和风格中性
        def neu_process(df):
            df = df.stack().reset_index()
            df.columns = ['date','ticker','factor']
            df = neturalize_ind(df,risk,risk_cols)
            resid = df.set_index(['date','ticker'])['resid'].unstack()
            return resid        
        factor_rank = neu_process(df)
        factor_rank = factor_rank.rank(axis=1,ascending=True) 
        
    return factor_rank

In [108]:
def neu_process(df):
    df = df.stack().reset_index()
    df.columns = ['date','ticker','factor']
    df = neturalize_ind(df,risk=risk,risk_cols=['logcmv','retsum120'])
    resid = df.set_index(['date','ticker'])['resid'].unstack()
    return resid    

In [109]:
def select_num_stocks(n):
    select_stocks = (factor_rank <= n)
    select_stocks_last = select_stocks.iloc[-1:,]
    select_stocks_last = select_stocks_last.replace(True,1).replace(False,np.nan)
    select_stocks_last = select_stocks_last.dropna(how='all',axis=1)
    return select_stocks_last

In [110]:
##市值排uni
mv_rank = risk1.set_index(['date','ticker'])['logcmv'].unstack()
mv_rank = mv_rank.rank(axis=1, ascending=True, pct=True)

##小市值
small = (mv_rank <= 0.33)
small = small.replace(True,1).replace(False,np.nan)
small = small.loc['2017':]

##中市值
median = (mv_rank <= 0.66) & (mv_rank >= 0.33)
median = median.replace(True,1).replace(False,np.nan)
median = median.loc['2017':]

##小中市值
sm = (mv_rank <= 0.66)
sm = sm.replace(True,1).replace(False,np.nan)
sm = sm.loc['2017':]

##大市值
large = (mv_rank>=0.66)
large = large.replace(True,1).replace(False,np.nan)
large = large.loc['2017':]

In [111]:
def position(df, period):
    empty = df > 0
    empty = empty.replace(True,np.nan).replace(False,np.nan)
    
    position = df.iloc[::period]
    position = pd.concat([empty, position])
    position = position.sort_values(['date'])
    position = position.reset_index()
    position = position.drop_duplicates(subset=['date'], keep='last').ffill(limit = period)
    position = position.set_index(['date'])
    return position

# 财务数据函数

In [112]:
def chg_format(df1):
    df1['ticker'] = df1['order_book_id'].str[:-5]
    df1['date'] = pd.to_datetime(df1['info_date'])
    df1['report_period'] = df1['quarter'].map(lambda x:x.replace('q1','0331').replace('q2','0630').replace('q3','0930').replace('q4','1231'))
    df1['report_period'] = df1['report_period'].astype('int')
    df1 = df1.sort_values(['date','ticker','report_period']).reset_index(drop=True)
    return df1

In [113]:
def func_save(factor,factor_name):
    ##只保留当前最新业绩的数据
    def func(x):
        x = x[x['report_period']>=x['report_period'].expanding(min_periods=1).max()]
        return x
    factor = factor.groupby('ticker').apply(func).reset_index(drop=True)
    #factor同一个date ticker 去重，保留当前最新report_period
    factor = factor.drop_duplicates(subset=['date','ticker'],keep='last')
    factor = factor.set_index(['date','ticker'])[factor_name].unstack()
    factor = pd.concat([factor,Open[['label']]],axis=1).drop(['label'],axis=1)
    factor = factor.fillna(method='ffill',limit=150).dropna(how='all',axis=0).dropna(how='all',axis=1)
    factor.to_pickle(os.path.join(path2,factor_name+'.pkl'))

In [114]:
##利润表流量单季度数据
dfaa = IS_pit
dfaa = dfaa.reset_index()
dfaa = chg_format(dfaa)
dfaa = dfaa.drop_duplicates(subset=['date','ticker','report_period'],keep='last')

In [115]:
#时间表
mls = pd.DataFrame(dfaa.report_period.unique())
mls.columns = ['report_period']
mls['pre_report_period'] = mls['report_period'].shift(1)
mls['pre2_report_period'] = mls['report_period'].shift(2)
mls['pre3_report_period'] = mls['report_period'].shift(3)
mls['next1_report_period'] = mls['report_period'].shift(-1)
mls['next2_report_period'] = mls['report_period'].shift(-2)
mls['next3_report_period'] = mls['report_period'].shift(-3)
mls = mls.replace(np.nan,0)
mls = mls.astype('int')

In [116]:
##现金流量单季度数据
dfbb = CF_pit
dfbb = dfbb.reset_index()
dfbb = chg_format(dfbb)
dfbb = dfbb.drop_duplicates(subset=['date','ticker','report_period'],keep='last')

In [117]:
#时间表
mlss = pd.DataFrame(dfbb.report_period.unique())
mlss.columns = ['report_period']
mlss['pre_report_period'] = mlss['report_period'].shift(1)
mlss['pre2_report_period'] = mlss['report_period'].shift(2)
mlss['pre3_report_period'] = mlss['report_period'].shift(3)
mlss['next1_report_period'] = mlss['report_period'].shift(-1)
mlss['next2_report_period'] = mlss['report_period'].shift(-2)
mlss['next3_report_period'] = mlss['report_period'].shift(-3)
mlss = mlss.replace(np.nan,0)
mlss = mlss.astype('int')

In [118]:
##债务截面单季度数据
dfcc = BS_pit
dfcc = dfcc.reset_index()
dfcc = chg_format(dfcc)
dfcc = dfcc.drop_duplicates(subset=['date','ticker','report_period'],keep='last')

In [119]:
#时间表
mlsss = pd.DataFrame(dfcc.report_period.unique())
mlsss.columns = ['report_period']
mlsss['pre_report_period'] = mlsss['report_period'].shift(1)
mlsss['pre2_report_period'] = mlsss['report_period'].shift(2)
mlsss['pre3_report_period'] = mlsss['report_period'].shift(3)
mlsss['next1_report_period'] = mlsss['report_period'].shift(-1)
mlsss['next2_report_period'] = mlsss['report_period'].shift(-2)
mlsss['next3_report_period'] = mlsss['report_period'].shift(-3)
mlsss = mlsss.replace(np.nan,0)
mlsss = mlsss.astype('int')

In [120]:
#IS,CF的单季度计算
def single_Q(df1,col):
    ##计算单季度数据：原理当季度数据发生更改时，当季的Q和下一季的Q会受到影响。
    ##分别计算当季的Q和下一季的Q 然后两个部分合并到一起。 要保证当季的Q和下一季的Q时，用到的数据日期全部不能大于当前日期。
    #part1：计算当季的Q
    df3 = df1[['date','ticker','quarter','report_period',col]].dropna(subset=[col])
    #拼上对应的前3个季度的report_period
    df3 = df3.merge(mls[['report_period','pre_report_period']],on=['report_period'],how='left')
    df3 = df3.merge(df3,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre'],how='left')
    df3 = df3.sort_values(['date','ticker','report_period','date_pre'])
    #单季数据=当期-上期 ，【第一季用当期】
    df3[col+'_Q'] = df3[col] - df3[col+'_pre']
    df3[col+'_Q'] = np.where(df3['quarter'].str[-2:] == 'q1',df3[col],df3[col+'_Q'])
    #保留所有Q1和除Q1之外date>=date_pre
    df3 = df3[(df3['quarter'].str[-2:]=='q1')|(df3['date']>=df3['date_pre'])] 
    df3 = df3.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #只保留date_pre最新的一条数据
    
    #part2：计算下一季Q
    tmp = df1[['date','ticker','quarter','report_period',col]].dropna(subset=[col])
    #拼上对应的前3个季度的report_period
    tmp = tmp.merge(mls[['report_period','next1_report_period']],on=['report_period'],how='left')
    tmp = tmp.merge(tmp,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],suffixes=['','_next1'],how='left')
    tmp[col+'_Q_next1'] = tmp[col+'_next1'] - tmp[col]
    tmp[col+'_Q_next1'] = np.where(tmp['quarter'].str[-2:] == 'q4',tmp[col+'_next1'],tmp[col+'_Q_next1']) #如果当前是四季度，
    #保留 next_quarter_date<=quarter_date
    tmp = tmp[tmp['date_next1']<=tmp['date']]
    tmp = tmp.drop_duplicates(subset=['date','ticker','report_period'],keep='last')  #只保留next_quarter_date<=quarter_date，且next_quarter_date最新的
    tmp['report_period'] = tmp['report_period_next1']
    tmp = tmp[(tmp['quarter'].str[-2:]!='q4')] #去掉所有当前为4季度的情况，如果当前为4季度，即便数据发成更改，也不影响下一季度的单季数据，因为下一季度是一季度
    tmp[col+'_Q'] = tmp[col+'_Q_next1']
    
    #把两个部分合并就是所有发生更新的数据
    part1 = df3[['date', 'ticker', 'report_period',col+'_Q']]
    part2 = tmp[['date', 'ticker', 'report_period',col+'_Q']]
    # part1['label'] = '1' #可删，只是为了识别数据来源
    # part2['label'] = '2' #可删，只是为了识别数据来源
    part12 = pd.concat([part1,part2])
    part12 = part12.sort_values(['date','ticker','report_period'])
    part12 = part12.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part12

In [121]:
#同比yoy
def compute_yoy(df4,col):#计算yoy
    tool_data = df4[['date','ticker','report_period',col]]
    #part1
    df1 = df4[['date', 'ticker', 'report_period',col]]
    df1['report_period'] = df1['report_period'].astype('int') ###新增
    df1['pre_yr_report_period'] = (df1['report_period'].astype('str').str[:4].astype('int')-1).astype('str') + df1['report_period'].astype('str').str[-4:]
    df1['pre_yr_report_period'] = df1['pre_yr_report_period'].astype('int')
    df1 = df1.merge(tool_data,left_on=['ticker','pre_yr_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_pre_yr'])
    df1 = df1.sort_values(['date','ticker','report_period','date_pre_yr'])
    df1 = df1[df1['date']>=df1['date_pre_yr']]
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_yoy'] = (df1[col]-df1[col+'_pre_yr'])/np.abs(df1[col+'_pre_yr'])
    #part2
    df2 = df4[['date', 'ticker', 'report_period',col]]
    df2['report_period'] = df2['report_period'].astype('int') ###新增
    df2['next_yr_report_period'] = (df2['report_period'].astype('str').str[:4].astype('int')+1).astype('str') + df2['report_period'].astype('str').str[-4:]
    df2['next_yr_report_period'] = df2['next_yr_report_period'].astype('int')
    df2 = df2.merge(tool_data,left_on=['ticker','next_yr_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_next_yr'])
    df2 = df2.sort_values(['date','ticker','report_period','date_next_yr'])
    df2 = df2[df2['date']>=df2['date_next_yr']]
    df2['report_period'] = df2['report_period_next_yr']
    df2 = df2.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df2[col+'_yoy'] = (df2[col+'_next_yr']-df2[col])/np.abs(df2[col])

    #把两个部分合并就是所有发生更新的数据
    part1 = df1[['date', 'ticker', 'report_period',col+'_yoy']]
    part2 = df2[['date', 'ticker', 'report_period',col+'_yoy']]
    part12 = pd.concat([part1,part2])
    part12 = part12.sort_values(['date','ticker','report_period'])
    part12 = part12.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part12

In [122]:
#T当季-去年同季
def compute_T(df4,col):
    tool_data = df4[['date','ticker','report_period',col]]
    #part1
    df1 = df4[['date', 'ticker', 'report_period',col]]
    df1['report_period'] = df1['report_period'].astype('int') ###新增
    df1['pre_yr_report_period'] = (df1['report_period'].astype('str').str[:4].astype('int')-1).astype('str') + df1['report_period'].astype('str').str[-4:]
    df1['pre_yr_report_period'] = df1['pre_yr_report_period'].astype('int')
    df1 = df1.merge(tool_data,left_on=['ticker','pre_yr_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_pre_yr'])
    df1 = df1.sort_values(['date','ticker','report_period','date_pre_yr'])
    df1 = df1[df1['date']>=df1['date_pre_yr']]
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_T'] = (df1[col]-df1[col+'_pre_yr'])
    #part2
    df2 = df4[['date', 'ticker', 'report_period',col]]
    df2['report_period'] = df2['report_period'].astype('int') ###新增
    df2['next_yr_report_period'] = (df2['report_period'].astype('str').str[:4].astype('int')+1).astype('str') + df2['report_period'].astype('str').str[-4:]
    df2['next_yr_report_period'] = df2['next_yr_report_period'].astype('int')
    df2 = df2.merge(tool_data,left_on=['ticker','next_yr_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_next_yr'])
    df2 = df2.sort_values(['date','ticker','report_period','date_next_yr'])
    df2 = df2[df2['date']>=df2['date_next_yr']]
    df2['report_period'] = df2['report_period_next_yr']
    df2 = df2.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df2[col+'_T'] = (df2[col+'_next_yr']-df2[col])

    #把两个部分合并就是所有发生更新的数据
    part1 = df1[['date', 'ticker', 'report_period',col+'_T']]
    part2 = df2[['date', 'ticker', 'report_period',col+'_T']]
    part12 = pd.concat([part1,part2])
    part12 = part12.sort_values(['date','ticker','report_period'])
    part12 = part12.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part12

In [123]:
#Tadd当季+去年同季
def compute_Tadd(df4,col):
    tool_data = df4[['date','ticker','report_period',col]]
    #part1
    df1 = df4[['date', 'ticker', 'report_period',col]]
    df1['report_period'] = df1['report_period'].astype('int') ###新增
    df1['pre_yr_report_period'] = (df1['report_period'].astype('str').str[:4].astype('int')-1).astype('str') + df1['report_period'].astype('str').str[-4:]
    df1['pre_yr_report_period'] = df1['pre_yr_report_period'].astype('int')
    df1 = df1.merge(tool_data,left_on=['ticker','pre_yr_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_pre_yr'])
    df1 = df1.sort_values(['date','ticker','report_period','date_pre_yr'])
    df1 = df1[df1['date']>=df1['date_pre_yr']]
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_T'] = (df1[col]+df1[col+'_pre_yr'])
    #part2
    df2 = df4[['date', 'ticker', 'report_period',col]]
    df2['report_period'] = df2['report_period'].astype('int') ###新增
    df2['next_yr_report_period'] = (df2['report_period'].astype('str').str[:4].astype('int')+1).astype('str') + df2['report_period'].astype('str').str[-4:]
    df2['next_yr_report_period'] = df2['next_yr_report_period'].astype('int')
    df2 = df2.merge(tool_data,left_on=['ticker','next_yr_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_next_yr'])
    df2 = df2.sort_values(['date','ticker','report_period','date_next_yr'])
    df2 = df2[df2['date']>=df2['date_next_yr']]
    df2['report_period'] = df2['report_period_next_yr']
    df2 = df2.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df2[col+'_T'] = (df2[col+'_next_yr']+df2[col])

    #把两个部分合并就是所有发生更新的数据
    part1 = df1[['date', 'ticker', 'report_period',col+'_T']]
    part2 = df2[['date', 'ticker', 'report_period',col+'_T']]
    part12 = pd.concat([part1,part2])
    part12 = part12.sort_values(['date','ticker','report_period'])
    part12 = part12.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part12

In [124]:
#环比QoQ
def compute_qoq(df4,col):
    tool_data = df4[['date','ticker','report_period',col]]
    #part1
    df1 = df4[['date','ticker','report_period','pre_report_period',col]]
    df1 = df1.merge(tool_data,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_pre'])
    df1 = df1.sort_values(['date','ticker','report_period','date_pre'])
    df1 = df1[df1['date']>=df1['date_pre']]
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_QoQ'] = (df1[col]-df1[col+'_pre'])/np.abs(df1[col+'_pre'])
    part1 = df1[['date','ticker','report_period',col+'_QoQ']]
    #part2
    df1 = df4[['date','ticker','report_period','next1_report_period',col]]
    df1 = df1.merge(tool_data,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_next'])
    df1 = df1.sort_values(['date','ticker','report_period','date_next'])
    df1 = df1[df1['date']>=df1['date_next']]
    df1['report_period'] = df1['report_period_next']
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_QoQ'] = (df1[col+'_next']-df1[col])/np.abs(df1[col])
    part2 = df1[['date','ticker','report_period',col+'_QoQ']]

    #把两个部分合并就是所有发生更新的数据
    part12 = pd.concat([part1,part2])
    part12 = part12.sort_values(['date','ticker','report_period'])
    part12 = part12.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part12

In [125]:
#D 当季-上季
def compute_D(df4,col):
    tool_data = df4[['date','ticker','report_period',col]]
    #part1
    df1 = df4[['date','ticker','report_period','pre_report_period',col]]
    df1 = df1.merge(tool_data,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_pre'])
    df1 = df1.sort_values(['date','ticker','report_period','date_pre'])
    df1 = df1[df1['date']>=df1['date_pre']]
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_D'] = df1[col]-df1[col+'_pre']
    part1 = df1[['date','ticker','report_period',col+'_D']]
    #part2
    df1 = df4[['date','ticker','report_period','next1_report_period',col]]
    df1 = df1.merge(tool_data,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_next'])
    df1 = df1.sort_values(['date','ticker','report_period','date_next'])
    df1 = df1[df1['date']>=df1['date_next']]
    df1['report_period'] = df1['report_period_next']
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_D'] = df1[col+'_next']-df1[col]
    part2 = df1[['date','ticker','report_period',col+'_D']]

    #把两个部分合并就是所有发生更新的数据
    part12 = pd.concat([part1,part2])
    part12 = part12.sort_values(['date','ticker','report_period'])
    part12 = part12.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part12

In [126]:
# 期初 + 期末 的均值
def compute_ChuMo(df4,col):
    tool_data = df4[['date','ticker','report_period',col]]
    #part1
    df1 = df4[['date','ticker','report_period','pre_report_period',col]]
    df1 = df1.merge(tool_data,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_pre'])
    df1 = df1.sort_values(['date','ticker','report_period','date_pre'])
    df1 = df1[df1['date']>=df1['date_pre']]
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_ChuMo'] = (df1[col+'_pre'] + df1[col])
    part1 = df1[['date','ticker','report_period',col+'_ChuMo']]
    #part2
    df1 = df4[['date','ticker','report_period','next1_report_period',col]]
    df1 = df1.merge(tool_data,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],how='left',suffixes=['','_next'])
    df1 = df1.sort_values(['date','ticker','report_period','date_next'])
    df1 = df1[df1['date']>=df1['date_next']]
    df1['report_period'] = df1['report_period_next']
    df1 = df1.drop_duplicates(subset=['date','ticker','report_period'],keep='last')   ##只保留'date','ticker','report_period'对应的'date_pre_yr'最新的一条数据
    df1[col+'_ChuMo'] = (df1[col] + df1[col+'_next'])
    part2 = df1[['date','ticker','report_period',col+'_ChuMo']]

    #把两个部分合并就是所有发生更新的数据
    part12 = pd.concat([part1,part2])
    part12 = part12.sort_values(['date','ticker','report_period'])
    part12 = part12.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part12

In [127]:
def compute_ttm(df4,col):
    #单季度数据计算的TTM，季度频率
    tool_data = df4[['date','ticker','report_period',col+'_Q']]
    #part1    
    df3 = df4.merge(tool_data,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','pre2_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre2'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','pre3_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre3'],how='left')
    #去掉 date_pre,date_pre2,date_pre3 大于date的行
    df3 = df3[(df3['date']>=df3['date_pre'])&(df3['date']>=df3['date_pre2'])&(df3['date']>=df3['date_pre3'])]
    ##只保留'date','ticker','report_period'对应的最新的一条数据
    df3 = df3.sort_values(['ticker','date','date_pre','date_pre2','date_pre3']).drop_duplicates(subset=['date','ticker','report_period'],keep='last')
    df3[col+'_TTM'] = df3[col+'_Q'] + df3[col+'_Q_pre'] + df3[col+'_Q_pre2']+ df3[col+'_Q_pre3']
    part1 = df3[['date', 'ticker', 'report_period',col+'_TTM']]
    #part1['label'] = 'part1'

    #part2
    df3 = df4.merge(tool_data,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','pre2_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre2'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],suffixes=['','_next1'],how='left')
    #去掉 date_pre,date_pre2,date_pre3 大于date的行
    df3 = df3[(df3['date']>=df3['date_pre'])&(df3['date']>=df3['date_pre2'])&(df3['date']>=df3['date_next1'])]
    df3['report_period'] = df3['report_period_next1']
    ##只保留'date','ticker','report_period'对应的最新的一条数据
    df3 = df3.sort_values(['ticker','date','date_pre','date_pre2','date_next1']).drop_duplicates(subset=['date','ticker','report_period'],keep='last')
    df3[col+'_TTM'] = df3[col+'_Q'] + df3[col+'_Q_pre'] + df3[col+'_Q_pre2']+ df3[col+'_Q_next1']
    part2 = df3[['date', 'ticker', 'report_period',col+'_TTM']]
    #part2['label'] = 'part2'

    #part3
    df3 = df4.merge(tool_data,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],suffixes=['','_next1'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','next2_report_period'],right_on=['ticker','report_period'],suffixes=['','_next2'],how='left')
    #去掉 date_pre,date_pre2,date_pre3 大于date的行
    df3 = df3[(df3['date']>=df3['date_pre'])&(df3['date']>=df3['date_next1'])&(df3['date']>=df3['date_next2'])]
    df3['report_period'] = df3['report_period_next2']
    ##只保留'date','ticker','report_period'对应的最新的一条数据
    df3 = df3.sort_values(['ticker','date','date_pre','date_next1','date_next2']).drop_duplicates(subset=['date','ticker','report_period'],keep='last')
    df3[col+'_TTM'] = df3[col+'_Q'] + df3[col+'_Q_pre'] + df3[col+'_Q_next1']+ df3[col+'_Q_next2']
    part3 = df3[['date', 'ticker', 'report_period',col+'_TTM']]
    #part3['label'] = 'part3'

    #part4
    df3 = df4.merge(tool_data,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],suffixes=['','_next1'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','next2_report_period'],right_on=['ticker','report_period'],suffixes=['','_next2'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','next3_report_period'],right_on=['ticker','report_period'],suffixes=['','_next3'],how='left')
    #去掉 date_pre,date_pre2,date_pre3 大于date的行
    df3 = df3[(df3['date']>=df3['date_next1'])&(df3['date']>=df3['date_next2'])&(df3['date']>=df3['date_next3'])]
    df3['report_period'] = df3['report_period_next3']
    ##只保留'date','ticker','report_period'对应的最新的一条数据
    df3 = df3.sort_values(['ticker','date','date_next1','date_next1','date_next1']).drop_duplicates(subset=['date','ticker','report_period'],keep='last')
    df3[col+'_TTM'] = df3[col+'_Q'] + df3[col+'_Q_next1'] + df3[col+'_Q_next2']+ df3[col+'_Q_next3']
    part4 = df3[['date', 'ticker', 'report_period',col+'_TTM']]
    #part4['label'] = 'part4'

    part1234 = pd.concat([part1,part2,part3,part4])
    part1234 = part1234.sort_values(['date','ticker','report_period'])
    part1234 = part1234.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part1234

In [128]:
def compute_ttm(df4,col):
    #单季度数据计算的TTM，季度频率
    tool_data = df4[['date','ticker','report_period',col+'_Q']]
    #part1    
    df3 = df4.merge(tool_data,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','pre2_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre2'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','pre3_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre3'],how='left')
    #去掉 date_pre,date_pre2,date_pre3 大于date的行
    df3 = df3[(df3['date']>=df3['date_pre'])&(df3['date']>=df3['date_pre2'])&(df3['date']>=df3['date_pre3'])]
    ##只保留'date','ticker','report_period'对应的最新的一条数据
    df3 = df3.sort_values(['ticker','date','date_pre','date_pre2','date_pre3']).drop_duplicates(subset=['date','ticker','report_period'],keep='last')
    df3[col+'_TTM'] = df3[col+'_Q'] + df3[col+'_Q_pre'] + df3[col+'_Q_pre2']+ df3[col+'_Q_pre3']
    part1 = df3[['date', 'ticker', 'report_period',col+'_TTM']]
    #part1['label'] = 'part1'

    #part2
    df3 = df4.merge(tool_data,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','pre2_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre2'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],suffixes=['','_next1'],how='left')
    #去掉 date_pre,date_pre2,date_pre3 大于date的行
    df3 = df3[(df3['date']>=df3['date_pre'])&(df3['date']>=df3['date_pre2'])&(df3['date']>=df3['date_next1'])]
    df3['report_period'] = df3['report_period_next1']
    ##只保留'date','ticker','report_period'对应的最新的一条数据
    df3 = df3.sort_values(['ticker','date','date_pre','date_pre2','date_next1']).drop_duplicates(subset=['date','ticker','report_period'],keep='last')
    df3[col+'_TTM'] = df3[col+'_Q'] + df3[col+'_Q_pre'] + df3[col+'_Q_pre2']+ df3[col+'_Q_next1']
    part2 = df3[['date', 'ticker', 'report_period',col+'_TTM']]
    #part2['label'] = 'part2'

    #part3
    df3 = df4.merge(tool_data,left_on=['ticker','pre_report_period'],right_on=['ticker','report_period'],suffixes=['','_pre'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],suffixes=['','_next1'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','next2_report_period'],right_on=['ticker','report_period'],suffixes=['','_next2'],how='left')
    #去掉 date_pre,date_pre2,date_pre3 大于date的行
    df3 = df3[(df3['date']>=df3['date_pre'])&(df3['date']>=df3['date_next1'])&(df3['date']>=df3['date_next2'])]
    df3['report_period'] = df3['report_period_next2']
    ##只保留'date','ticker','report_period'对应的最新的一条数据
    df3 = df3.sort_values(['ticker','date','date_pre','date_next1','date_next2']).drop_duplicates(subset=['date','ticker','report_period'],keep='last')
    df3[col+'_TTM'] = df3[col+'_Q'] + df3[col+'_Q_pre'] + df3[col+'_Q_next1']+ df3[col+'_Q_next2']
    part3 = df3[['date', 'ticker', 'report_period',col+'_TTM']]
    #part3['label'] = 'part3'

    #part4
    df3 = df4.merge(tool_data,left_on=['ticker','next1_report_period'],right_on=['ticker','report_period'],suffixes=['','_next1'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','next2_report_period'],right_on=['ticker','report_period'],suffixes=['','_next2'],how='left')
    df3 = df3.merge(tool_data,left_on=['ticker','next3_report_period'],right_on=['ticker','report_period'],suffixes=['','_next3'],how='left')
    #去掉 date_pre,date_pre2,date_pre3 大于date的行
    df3 = df3[(df3['date']>=df3['date_next1'])&(df3['date']>=df3['date_next2'])&(df3['date']>=df3['date_next3'])]
    df3['report_period'] = df3['report_period_next3']
    ##只保留'date','ticker','report_period'对应的最新的一条数据
    df3 = df3.sort_values(['ticker','date','date_next1','date_next1','date_next1']).drop_duplicates(subset=['date','ticker','report_period'],keep='last')
    df3[col+'_TTM'] = df3[col+'_Q'] + df3[col+'_Q_next1'] + df3[col+'_Q_next2']+ df3[col+'_Q_next3']
    part4 = df3[['date', 'ticker', 'report_period',col+'_TTM']]
    #part4['label'] = 'part4'

    part1234 = pd.concat([part1,part2,part3,part4])
    part1234 = part1234.sort_values(['date','ticker','report_period'])
    part1234 = part1234.drop_duplicates(subset=['date','ticker','report_period'],keep='last') #删掉重复行
    return part1234

In [129]:
def process_finfactor_and_save_data(df, col3, Open):
    # 去重
    df_processed = df.drop_duplicates(subset=['date', 'ticker'], keep='last')
    
    # 设置索引并unstack
    df_processed = df_processed.set_index(['date', 'ticker'])[col3].unstack()
    
    # 合并标签数据并删除标签列
    df_processed = pd.concat([df_processed, Open[['label']]], axis=1).drop(['label'], axis=1)
    
    # 填充缺失值并删除全为NaN的行和列
    df_processed = df_processed.fillna(method='ffill', limit=150).dropna(how='all', axis=0).dropna(how='all', axis=1)
    return df_processed

In [130]:
def filter_extreme_by_MAD(series, n=3):
    median = series.median()
    median_new = abs(series - median).median()
    max_value = median + n * median_new
    min_value = median - n * median_new
    return np.clip(series, min_value, max_value, axis=1)

In [131]:
 def func(x):
        x = x[x['report_period']>=x['report_period'].expanding(min_periods=1).max()]
        return x

# 分钟量价

In [ ]:
#导入数据 
def k1min_data_process(date_str):
    k1min= oss_client.read_oss_parquet_file(f'ad_hoc_prod/1min_cs/{date_str}.parquet')
    k1min = k1min.sort_values(by=["RIC", "TIME"])
    K1min = k1min.reset_index(drop=True)
    k1min['RETURN'] = k1min['CLOSE']/k1min['OPEN'] - 1
    k1min = k1min.rename(columns={'RIC': 'ticker', 'DATE': 'date'})
    return k1min

In [6]:
# 最大20vulume的mmt
def mmt_top20_volume(single_day_data):

    ### 公式函数
    df = (
        single_day_data.assign(vol_rank=lambda x: x.groupby(['ticker', 'date'])['VOL']
        .transform(lambda s: s.rank(method='first', ascending=False))).query('vol_rank <= 20')
        .groupby(['ticker', 'date'])['RETURN']
        .sum()
        .reset_index()
        .drop(columns=['vol_rank'], errors='ignore')
    )
    ### 公式函数
    df = pd.DataFrame(df)
    df = df.reset_index()
    df = df.rename(columns={df.columns[-1]: 'factor'})
    df = df.pivot(index='date', columns='ticker', values='factor')
    return df
    
##多天   
start_date = '2017-01-03'
end_date = '2025-01-04'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

final_df = pd.DataFrame()

for date in date_range:
    date_str = date.date().strftime('%Y-%m-%d')
 
    try:
        single_day_data = k1min_data_process(date_str)

        ### 公式函数
        cal_factor = mmt_top20_volume(single_day_data)
        ### 公式函数

        final_df = pd.concat([final_df, cal_factor])
        
    except NoSuchKey:
        # 捕获 NoSuchKey 异常并跳过该日期
        print(f"File for date {date_str} does not exist. Skipping...")
        continue
        
final_df = -1*final_df 
final_df.to_pickle('D:/redata/factor/mmt_last30min@factor.pkl') #如果错过了读这个文件先

## 单天
date = "2025-02-20"
mmt_top20_vol_today = mmt_top20_volume(k1min)
mmt_top20_vol_full_data = pd.read_pickle('D:/redata/factor/mmt_top20_vol@factor.pkl')
mmt_top20_vol_full_data =  pd.concat([mmt_top20_vol_full_data, mmt_top20_vol_today])
mmt_top20_vol_full_data.to_pickle(f'D:/redata/factor/mmt_top20_vol{date}@factor.pkl')

mmt_top20volumeRet_std_W = -1 * mmt_top20_vol_full_data.rolling(5).std()
mmt_top20volumeRet_std_W.to_pickle('D:/redata/factor/mmt_top20volumeRet_std_W@factor.pkl')

File for date 2017-01-01 does not exist. Skipping...
File for date 2017-01-02 does not exist. Skipping...
File for date 2017-01-07 does not exist. Skipping...
File for date 2017-01-08 does not exist. Skipping...
File for date 2017-01-14 does not exist. Skipping...
File for date 2017-01-15 does not exist. Skipping...
File for date 2017-01-21 does not exist. Skipping...
File for date 2017-01-22 does not exist. Skipping...
File for date 2017-01-27 does not exist. Skipping...
File for date 2017-01-28 does not exist. Skipping...
File for date 2017-01-29 does not exist. Skipping...
File for date 2017-01-30 does not exist. Skipping...
File for date 2017-01-31 does not exist. Skipping...
File for date 2017-02-01 does not exist. Skipping...
File for date 2017-02-02 does not exist. Skipping...
File for date 2017-02-04 does not exist. Skipping...
File for date 2017-02-05 does not exist. Skipping...
File for date 2017-02-11 does not exist. Skipping...
File for date 2017-02-12 does not exist. Skipp

KeyboardInterrupt: 

In [ ]:
#尾盘半小时动量
def mmt_last30min(single_day_data):
    
    ### 公式函数
    df = single_day_data.groupby('ticker').tail(31)
    df = df.groupby(['ticker', 'date'])['CLOSE'].apply(lambda x: x.iloc[-1] / x.iloc[0] - 1)
    ### 公式函数
    
    df = pd.DataFrame(df)
    df = df.reset_index()
    
    df = df.pivot(index='date', columns='ticker', values='CLOSE')
    return df

##多天   
start_date = '2017-01-03'
end_date = '2025-01-04'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

final_df = pd.DataFrame()

for date in date_range:
    date_str = date.date().strftime('%Y-%m-%d')
 
    try:
        single_day_data = k1min_data_process(date_str)

        ### 公式函数
        cal_factor = mmt_last30min(single_day_data)
        ### 公式函数

        final_df = pd.concat([final_df, cal_factor])
        
    except NoSuchKey:
        # 捕获 NoSuchKey 异常并跳过该日期
        print(f"File for date {date_str} does not exist. Skipping...")
        continue
        
final_df = -1*final_df 
final_df.to_pickle('D:/redata/factor/mmt_last30min@factor.pkl') #如果错过了读这个文件先

## 单天
date = "2025-02-20"
mmt_last30min_today = mmt_last30min(k1min_data_process(date))
mmt_last30min_full_data = pd.read_pickle('D:/redata/factor/mmt_last30min@factor.pkl')
mmt_last30min_full_data =  pd.concat([mmt_last30min_full_data, mmt_last30min_today])
mmt_last30min_full_data.to_pickle(f'D:/redata/factor/mmt_last30min_{date}@factor.pkl')

mmt_last30min_std = -1 * mmt_last30min_full_data.rolling(20).std()
mmt_last30min_std.to_pickle('D:/redata/factor/mmt_last30min_std@factor.pkl')

In [134]:
# # 计算每50根K线中50个beta值的均值
# window = 50
# average_betas = []

# for i in range(0, len(k1min) - window + 1, window):  # 步长为50，滑动窗口
#     betas = []
#     for j in range(i, i + window):  # 在每50根K线内计算beta
#         y = k1min[['HIGH']].iloc[j:j + window].values.reshape(-1, 1)
#         x = k1min[['LOW']].iloc[j:j + window].values.reshape(-1, 1)
        
#         # 检查是否足够的数据点
#         if len(y) == window and len(x) == window:
#             model = LinearRegression().fit(x, y)
#             beta = model.coef_[0][0]
#             betas.append(beta)
    
#     # 计算当前50根K线的beta均值
#     if betas:
#         average_beta = np.mean(betas)
#         average_betas.append(average_beta)

# # 输出每50根K线的50个beta均值
# print(average_betas)

In [135]:
# def rolling_regression(high, low):
#     X = high.values.reshape(-1, 1)
#     y = low.values.reshape(-1, 1)
#     model = LinearRegression().fit(X, y)
#     return model.coef_[0][0]

# # Apply the rolling regression function to the grouped DataFrame
# result = k1min.groupby(['ticker', 'date'])[['HIGH', 'LOW']].rolling(50).apply(
#     lambda x: rolling_regression(x['HIGH'], x['LOW'])
# )

# # Reset the index to make it more readable
# result = result.reset_index(level=[0, 1], drop=True)

# # Display the result
# print(result)

In [136]:
# QRS支撑阻力指标
# def mmt_ols_beta_mean(data_1min):
    
#     ### 公式函数
#     y = data_1min['HIGH'].values.reshape(-1,1)
#     x = data_1min['LOW'].values.reshape(-1,1)
#     model = LinearRegression().fit(x, y)
#     beta = model.coef_[0][0]
    
#     factor = factor.groupby(['ticker', 'date'])['CLOSE'].apply(lambda x: x.iloc[-1] / x.iloc[0] - 1)
#     ### 公式函数
    
#     factor = pd.DataFrame(factor)
#     factor.columns = [date_data]
#     factor = factor.reset_index()
    
#     pivot_df = factor.pivot(index='date', columns='ticker', values=date_data)
#     pivot_df = pivot_df * -1
#     return pivot_df

# factor_today = mmt_ols_beta_mean(k1min)
# factor_full_data = pd.read_pickle('D:/redata/factor/mmt_ols_beta_mean@factor.pkl')
# factor_full_data =  pd.concat([factor_full_data, factor_today])
# factor_full_data.to_pickle('D:/redata/factor/mmt_ols_beta_mean@factor.pkl')

# 小市值因子


In [137]:
#流动市值对数
maketcap = -1 * np.log(adj_close * total_a)
maketcap.to_pickle('D:/redata/factor/maketcap@factor.pkl')
#收盘价
adjclose = adj_close * -1
adjclose.to_pickle('D:/redata/factor/adjclose@factor.pkl')

In [138]:
#动量
mmt_overnight_M = ((adj_open - adj_close.shift(1))/adj_close.shift(1)).rolling(20).sum()
mmt_overnight_M.to_pickle('D:/redata/factor/mmt_overnight_M@factor.pkl')

mmt_overnight_A = ((adj_open - adj_close.shift(1))/adj_close.shift(1)).rolling(240).sum() - ((adj_open - adj_close.shift(1))/adj_close.shift(1)).rolling(20).sum()
mmt_overnight_A.to_pickle('D:/redata/factor/mmt_overnight_A@factor.pkl')

mmt_normal_M = -(adj_open/adj_open.shift(20) - 1)
mmt_normal_M.to_pickle('D:/redata/factor/mmt_normal_M@factor.pkl')

mmt_normal_A = -(adj_open/adj_open.shift(240) - 1)
mmt_normal_A.to_pickle('D:/redata/factor/mmt_normal_A@factor.pkl')

# mmt_report_overnight = 
# mmt_report_jump_open
# mmt_report_period

mmt_intraday_M = ((adj_open - adj_close)/adj_close).rolling(20).sum()
mmt_intraday_M.to_pickle('D:/redata/factor/mmt_intraday_M@factor.pkl')

mmt_avg_M = -(adj_close/adj_close.rolling(20).mean())
mmt_avg_M.to_pickle('D:/redata/factor/mmt_avg_M@factor.pkl')

In [139]:
mmt_overnight_A.tail(1)

ticker,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,688787,688788,688789,688793,688798,688799,688800,688819,688981,689009
date,,,,,,,,,,,,,,,,,,,,,
2024-12-27,-0.043227,-0.550392,-1.219713,NaN,-0.284113,-0.022479,-0.159682,0.091171,-0.399197,-0.427065,...,0.222258,0.404464,0.123911,-0.078095,0.269668,0.114746,0.326303,0.346499,0.074029,0.316789


In [140]:
limit_1 = (close == limit_down).astype(int)
limit_2 = (close == limit_up).astype(int)
limit_3 = limit_1+limit_2
limit = limit_3.replace({0: 1, 1: 0})
adj_close_noZDT = adj_close*limit

mmt_off_limit_M = -(adj_close_noZDT - adj_close_noZDT.shift(20))/adj_close_noZDT.shift(20)
mmt_off_limit_M = mmt_off_limit_M.replace(1,np.nan)
mmt_off_limit_M.to_pickle('D:/redata/factor/mmt_off_limit_M@factor.pkl')

mmt_off_limit_A = (adj_close_noZDT - adj_close_noZDT.shift(240))/adj_close_noZDT.shift(240)
mmt_off_limit_A = mmt_off_limit_A.replace(np.inf,np.nan).replace(-np.inf,np.nan)
mmt_off_limit_A = mmt_off_limit_A.replace(-1,np.nan)
mmt_off_limit_A = (mmt_off_limit_A - mmt_off_limit_M)
mmt_off_limit_A = -1 * mmt_off_limit_A
mmt_off_limit_A.to_pickle('D:/redata/factor/mmt_off_limit_A@factor.pkl')

In [141]:
# mmt_range_M
amp = adj_high - adj_low
rnt = adj_return
empty = adj_return -adj_return
empty = empty.replace(0,np.nan)
rows = len(amp) -20

for i in range(0,rows):
    amp_20d = amp.iloc[i:i+20,:]
    rnt_20d = rnt.iloc[i:i+20,:]

    amp_df = amp_20d.rank(ascending=False,method = 'min')

    amp_top20 = (amp_df <= 4).astype(int)
    amp_low20 = (amp_df >= 16).astype(int)

    rnt_top20 = (rnt_20d*amp_top20).rolling(20).sum()
    rnt_low20 = (rnt_20d*amp_low20).rolling(20).sum()
    result = rnt_top20 - rnt_low20
    empty.iloc[i+20:i+21,:] = result.iloc[-1:,:]

mmt_range_M = -1 * empty
mmt_range_M.to_pickle('D:/redata/factor/mmt_range_M@factor.pkl')

In [142]:
#波动率
vol_up_std_6M = -1 * adj_return.rolling(120).std()
vol_up_std_6M.to_pickle('D:/redata/factor/vol_up_std_6M@factor.pkl')

vol_highlow_std_6M = -1 * (adj_high/adj_low).rolling(120).std()
vol_highlow_std_6M.to_pickle('D:/redata/factor/vol_highlow_std_6M@factor.pkl')

vol_upshadow_std_6M = -1* ((adj_high-np.maximum(adj_open,adj_close))/adj_high).rolling(120).std()
vol_upshadow_std_6M.to_pickle('D:/redata/factor/vol_upshadow_std_6M@factor.pkl')

vol_w_downshadow_std_6M = -1 * ((adj_close-adj_low)/adj_low).rolling(120).std()
vol_w_downshadow_std_6M.to_pickle('D:/redata/factor/vol_w_downshadow_std_6M@factor.pkl')

In [143]:
#流动性
liq_turn_std_6M = -1* turnover_rate.rolling(120).std()
liq_turn_std_6M.to_pickle('D:/redata/factor/liq_turn_std_6M@factor.pkl')

liq_vstd_1M = -1* total_turnover.rolling(20).sum()/adj_return.rolling(20).std()
liq_vstd_1M.to_pickle('D:/redata/factor/liq_vstd_1M@factor.pkl')

liq_amiihud_avg_1M = (np.abs(adj_return)/total_turnover).rolling(20).mean()
liq_amiihud_avg_1M.to_pickle('D:/redata/factor/liq_amiihud_avg_1M@factor.pkl')

liq_shortcut_avg_1M = ((2*(adj_high -adj_low) - np.abs(adj_open -adj_close))/total_turnover).rolling(20).mean()
liq_shortcut_avg_1M.to_pickle('D:/redata/factor/liq_shortcut_avg_1M@factor.pkl')

In [144]:
#量价cov
corr_ret_turnd_1M = turnover_rate.rolling(20).cov(adj_return)
corr_ret_turnd_1M.to_pickle('D:/redata/factor/corr_ret_turnd_1M@factor.pkl')

corr_price_turn_1M = -1* turnover_rate.rolling(20).cov(adj_close)
corr_price_turn_1M.to_pickle('D:/redata/factor/corr_price_turn_1M@factor.pkl')

corr_ret_turn_post_1M = -1 * turnover_rate.shift(1).rolling(20).cov(adj_close)
corr_ret_turn_post_1M.to_pickle('D:/redata/factor/corr_ret_turn_post_1M@factor.pkl')

In [145]:
#复合
mom_A = 0.7*mmt_off_limit_A + 0.3*mmt_normal_A
mom_M = 0.7*mmt_off_limit_M + 0.3*mmt_normal_M
mom_overnight_A = mmt_overnight_A
reverse_M = 0.5*mmt_intraday_M + 0.5*mmt_range_M
vol_6M = 0.25*vol_up_std_6M + 0.25*vol_highlow_std_6M + 0.25*vol_upshadow_std_6M + 0.25*vol_w_downshadow_std_6M
turnrate_6M = liq_turn_std_6M
ill_price_M = 0.33333*liq_vstd_1M + 0.33333*liq_amiihud_avg_1M + 0.33333*liq_shortcut_avg_1M
corrPV_M = 0.33333*corr_ret_turnd_1M + 0.33333*corr_price_turn_1M + 0.33333*corr_ret_turn_post_1M

mom_A.to_pickle('D:/redata/factor/mom_A@factor.pkl')
mom_M.to_pickle('D:/redata/factor/mom_M@factor.pkl')
mom_overnight_A.to_pickle('D:/redata/factor/mom_overnight_A@factor.pkl')
reverse_M.to_pickle('D:/redata/factor/reverse_M@factor.pkl')
vol_6M.to_pickle('D:/redata/factor/vol_6M@factor.pkl')
turnrate_6M.to_pickle('D:/redata/factor/turnrate_6M@factor.pkl')
ill_price_M.to_pickle('D:/redata/factor/ill_price_M@factor.pkl')
corrPV_M.to_pickle('D:/redata/factor/corrPV_M@factor.pkl')

In [146]:
#技术
volume_20d_fac = -1 * volume.rolling(20).std()/volume.rolling(20).mean()
volume_60d_ratio = -1 * volume.rolling(60).mean()/volume.rolling(240).mean()

volume_20d_fac.to_pickle('D:/redata/factor/volume_20d_fac@factor.pkl')
volume_60d_ratio.to_pickle('D:/redata/factor/volume_60d_ratio@factor.pkl')

# 量价

## 稳定因子

换手率

In [147]:
#STR 凸显度
df_ret = adj_return.copy()

def sigma_cal(df_ret, theta=0.1):
    returns_median = df_ret.median(axis=1)
    frac1 = df_ret.sub(returns_median,axis=0).abs()
    frac2 = df_ret.abs().add(returns_median.abs(), axis=0) + theta
    sigma = frac1.div(frac2)
    return sigma
    
def weight_cal(sigma, time_range=20, delta=0.7):
    df = sigma.groupby(pd.Grouper(freq='20D'))
    # df_cleaned = df.dropna(axis=1,how='any')
    df_rank = df.apply(lambda x: x.rank(axis=0, ascending=False))
    df_rank = df_rank.reset_index(level=0, drop=True)
    frac1= df_rank.apply(lambda x: np.power(delta, x))
    frac2= frac1.mean(axis=1)
    weight= frac1.div(frac2, axis=0)
    # assert frac1.iloc[0,1] / frac2.iloc[0] == weight.iloc[0,1]
    return weight

sigma = sigma_cal(df_ret,theta=1)
weight = weight_cal(sigma, time_range=20, delta=0.7)
STR = -1*weight.rolling(20).cov(df_ret)

STR = STR.rolling(20).mean()
STR.to_pickle('D:/redata/factor/STR@factor.pkl')

In [148]:
#改进惊恐因子
ret_daily = adj_return.copy()

idx000985['idx985_ret'] = idx000985['close']/idx000985['prev_close'] -1
idx000985 = idx000985[(idx000985['date']>='2014-01-02')]
ret_daily['idx985_ret'] = idx000985['idx985_ret'].values

def sigma_cal(ret_daily, theta=0.1):
    idx985 = ret_daily.iloc[:,-1]
    frac1 = ret_daily.sub(idx985,axis=0).abs()
    frac2 = ret_daily.abs().add(idx985.abs(), axis=0) + theta
    sigma = frac1.div(frac2)
    return sigma
terra = sigma_cal(ret_daily, theta=0.1)
terra = terra.drop(columns=terra.columns[-1])

weighted_dec = ret_daily * terra
mean_20terra = weighted_dec.rolling(20).mean()
std_20terra = weighted_dec.rolling(20).std()
terra_20d = -(mean_20terra + std_20terra)

terra_20d.to_pickle('D:/redata/factor/terra_20d@factor.pkl')

换手率UMR

收益率

In [149]:
wt = 0.5 ** (1/40)
daily_standard_deviation = -1*(np.sqrt(wt * ((adj_return - adj_return.rolling(252).mean())**2).rolling(252).sum()))

daily_standard_deviation.to_pickle('D:/redata/factor/daily_standard_devijjwdation@factor.pkl')

In [150]:
ret_daily = adj_return.copy()
idx985 = idx000985.copy()
idx985['idx985_ret'] = idx985['close']/idx985['prev_close'] -1
idx985 = idx985[(idx985['date']>='2014-01-02')].set_index('date')
idx985 = idx985[['idx985_ret']]
# 计算超额收益
excess_ret = ret_daily.sub(idx985['idx985_ret'], axis=0)

In [151]:
high1 = adj_high
low1 = adj_low
close1 = adj_close

high_r_std_Nm = (high1 / close1.shift(1)).rolling(120).std()
low_r_std_Nm = (low1 / close1.shift(1)).rolling(120).std()

hml_r_std_Nm = (high_r_std_Nm - low_r_std_Nm) * -1
hpl_r_std_Nm = (high_r_std_Nm + low_r_std_Nm) * -1

# hml_r_std_Nm = filter_extreme_by_MAD(hml_r_std_Nm)
# hpl_r_std_Nm = filter_extreme_by_MAD(hpl_r_std_Nm)

hml_r_std_Nm.to_pickle('D:/redata/factor/hml_r_std_6m@factor.pkl')
hpl_r_std_Nm.to_pickle('D:/redata/factor/hpl_r_std_6m@factor.pkl')

high_r_std_N = (high1 / close1.shift(1)).rolling(100).std()
high_r_std_N = -1 * high_r_std_Nm
# high_r_std_N = filter_extreme_by_MAD(high_r_std_N)
high_r_std_N.to_pickle('D:/redata/factor/high_r_std_5m@factor.pkl')

In [152]:
ma_5 = adj_close.rolling(5).mean() * -1
bias_5 = -1*((adj_close/ma_5) - 1)
bias_5.to_pickle('D:/redata/factor/bias_5@factor.pkl')

# 300因子

In [153]:
#Mon12m
adj_return_daily = adj_return.fillna(0)
monthly_returns = adj_return_daily.groupby(adj_return_daily.index.to_period('M')).apply(lambda x: (1 + x).prod() - 1)  # 累积每日收益率
monthly_returns = monthly_returns +1
monthly_returns = monthly_returns.rolling(11).apply(lambda x: (x).prod())
monthly_returns = monthly_returns -1

empty = adj_close - adj_close
empty = empty.replace(0,np.nan)
empty.iloc[-1:,:] = monthly_returns.iloc[-1:,:]
mon12m = empty.bfill()
mon12m = mon12m.fillna(0)
mon12m.to_pickle('D:/redata/factor/mon12m@factor.pkl')

In [154]:
empty = adj_close - adj_close
empty = empty.replace(0,np.nan)
empty.iloc[-1:,:] = monthly_returns.iloc[-1:,:]
mon12m = empty.bfill()
mon12m

ticker,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,688787,688788,688789,688793,688798,688799,688800,688819,688981,689009
date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,0.368541,-0.213542,0.17247,-0.117021,0.923261,0.588367,0.455399,-0.122828,0.138211,0.091957,...,1.149057,0.178856,0.261117,0.166018,0.385387,0.155855,1.124046,0.163362,1.252483,1.020991
2014-01-03,0.368541,-0.213542,0.17247,-0.117021,0.923261,0.588367,0.455399,-0.122828,0.138211,0.091957,...,1.149057,0.178856,0.261117,0.166018,0.385387,0.155855,1.124046,0.163362,1.252483,1.020991
2014-01-06,0.368541,-0.213542,0.17247,-0.117021,0.923261,0.588367,0.455399,-0.122828,0.138211,0.091957,...,1.149057,0.178856,0.261117,0.166018,0.385387,0.155855,1.124046,0.163362,1.252483,1.020991
2014-01-07,0.368541,-0.213542,0.17247,-0.117021,0.923261,0.588367,0.455399,-0.122828,0.138211,0.091957,...,1.149057,0.178856,0.261117,0.166018,0.385387,0.155855,1.124046,0.163362,1.252483,1.020991
2014-01-08,0.368541,-0.213542,0.17247,-0.117021,0.923261,0.588367,0.455399,-0.122828,0.138211,0.091957,...,1.149057,0.178856,0.261117,0.166018,0.385387,0.155855,1.124046,0.163362,1.252483,1.020991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-23,0.368541,-0.213542,0.17247,-0.117021,0.923261,0.588367,0.455399,-0.122828,0.138211,0.091957,...,1.149057,0.178856,0.261117,0.166018,0.385387,0.155855,1.124046,0.163362,1.252483,1.020991
2024-12-24,0.368541,-0.213542,0.17247,-0.117021,0.923261,0.588367,0.455399,-0.122828,0.138211,0.091957,...,1.149057,0.178856,0.261117,0.166018,0.385387,0.155855,1.124046,0.163362,1.252483,1.020991
2024-12-25,0.368541,-0.213542,0.17247,-0.117021,0.923261,0.588367,0.455399,-0.122828,0.138211,0.091957,...,1.149057,0.178856,0.261117,0.166018,0.385387,0.155855,1.124046,0.163362,1.252483,1.020991


In [155]:
#AM

#单季度
col1 = 'total_assets'
df_single_Q = single_Q(dfcc,col1)

col2 = 'total_assets_Q'
df_fin = process_finfactor_and_save_data(df_single_Q, col2, open)

AM = df_fin / circulation_market_value
AM.to_pickle('D:/redata/factor/AM@factor.pkl')

In [156]:
#cashprod

#单季度
col1 = 'total_assets'
df_single_Q = single_Q(dfcc,col1)

col2 = 'total_assets_Q'
at = process_finfactor_and_save_data(df_single_Q, col2, open)

#单季度
col11 = 'cash_equivalent'
df_single_Qq = single_Q(dfcc,col11)

col22 = 'cash_equivalent_Q'
che = process_finfactor_and_save_data(df_single_Qq, col22, open)

cashprod = (circulation_market_value - at)/che
cashprod.to_pickle('D:/redata/factor/cashprod@factor.pkl')

In [157]:
#Gross profitability

#单季度
col1 = 'total_assets'
df_single_Q = single_Q(dfcc,col1)

col2 = 'total_assets_Q'
at = process_finfactor_and_save_data(df_single_Q, col2, open)

#单季度
col11 = 'cost_of_goods_sold'
df_single_Qq = single_Q(dfaa,col11)

col22 = 'cost_of_goods_sold_Q'
cogs = process_finfactor_and_save_data(df_single_Qq, col22, open)

#单季度
col111 = 'revenue'
df_single_Qqq = single_Q(dfaa,col111)

col222 = 'revenue_Q'
rvn = process_finfactor_and_save_data(df_single_Qqq, col222, open)

gp = (rvn - cogs)/at
gp.to_pickle('D:/redata/factor/gp@factor.pkl')

In [158]:
#dolvol

dolvol = np.log(volume.shift(60)*np.abs(adj_close.shift(60)))
dolvol = -dolvol
dolvol.to_pickle('D:/redata/factor/dolvol@factor.pkl')

In [159]:
#Eearnig2Price

#单季度
col1 = 'profit_before_tax'
df_singleQ = single_Q(dfaa,col1)

col2 = 'profit_before_tax_Q'
df_fin = process_finfactor_and_save_data(df_singleQ, col2, open)

EP = df_fin / circulation_market_value
# EP = np.where(EP < 0, np.nan, EP)
a = EP>0
Eearnig2Price = (EP*a).replace(0,np.nan)
Eearnig2Price.to_pickle('D:/redata/factor/Eearnig2Price@factor.pkl')

In [160]:
#52weekshigh

temp = adj_close.rolling(window=240, min_periods=20).max()
h52 = adj_close/ temp
h52.to_pickle('D:/redata/factor/h52@factor.pkl')                  

In [161]:
#ill
ill = np.abs(adj_return)/(adj_close*volume)
ill = ill.rolling(20).mean()


# 财务因子

## 稳定因子

In [162]:
#净利润同比增速
#单季度
col1 = 'net_profit'
df_single_Q = single_Q(dfaa,col1)
df_merge = df_single_Q.merge(mls,on=['report_period'],how='left')

# ttm12月累计——QoQ环比增速——yoy同比增速——Diff当季-上季数值
col2 = 'net_profit_Q'
df_computed = compute_yoy(df_merge,col2)

col3 = 'net_profit_Q_yoy'
df_fin = process_finfactor_and_save_data(df_computed, col3, open)
df_fin.to_pickle('D:/redata/factor/net_profit_Q_yoy@factor.pkl')

In [163]:
#营业利润同比增速
#单季度
col1 = 'operating_revenue'
df_single_Q = single_Q(dfaa,col1)
df_merge = df_single_Q.merge(mls,on=['report_period'],how='left')

# ttm12月累计——QoQ环比增速——yoy同比增速——Diff当季-上季数值
col2 = 'operating_revenue_Q'
df_computed = compute_yoy(df_merge,col2)

col3 = 'operating_revenue_Q_yoy'
df_fin = process_finfactor_and_save_data(df_computed, col3, open)
df_fin.to_pickle('D:/redata/factor/operating_revenue_Q_yoy@factor.pkl')

In [164]:
#归母净利润同比增速

#单季度
col1 = 'net_profit_parent_company'
df_single_Q = single_Q(dfaa,col1)
df_merge = df_single_Q.merge(mls,on=['report_period'],how='left')

# ttm12月累计——QoQ环比增速——yoy同比增速——Diff当季-上季数值
col2 = 'net_profit_parent_company_Q'
df_computed = compute_yoy(df_merge,col2)

col3 = 'net_profit_parent_company_Q_yoy'
df_fin = process_finfactor_and_save_data(df_computed, col3, open)
df_fin.to_pickle('D:/redata/factor/net_profit_parent_company_Q_yoy@factor.pkl')

In [165]:
# 当季营业利润同比增长率

#单季度
col1 = 'operating_revenue'
df_single_Q = single_Q(dfaa,col1)
df_merge = df_single_Q.merge(mls,on=['report_period'],how='left')

# ttm12月累计——QoQ环比增速——yoy同比增速——Diff当季-上季数值
col2 = 'operating_revenue_Q'
df_computed = compute_yoy(df_merge,col2)

col3 = 'operating_revenue_Q_yoy'
df_fin = process_finfactor_and_save_data(df_computed, col3, open)
df_fin.to_pickle('D:/redata/factor/operating_revenue_Q_yoy@factor.pkl')

In [166]:
# 营业利润ttm

#单季度
col1 = 'operating_revenue'
df_single_Q = single_Q(dfaa,col1)
df_merge = df_single_Q.merge(mls,on=['report_period'],how='left')

# ttm12月累计——QoQ环比增速——yoy同比增速——Diff当季-上季数值
col2 = 'operating_revenue'
df_computed = compute_ttm(df_merge,col2)

col3 = 'operating_revenue_TTM'
df_fin = process_finfactor_and_save_data(df_computed, col3, open)

sp_ttm = df_fin / mc
sp_ttm.to_pickle('D:/redata/factor/sp_ttm@factor.pkl')

In [167]:
# BP 股东权益/总市值

#单季度
col1 = 'equity_parent_company'
df_singleQ = single_Q(dfcc,col1)

col2 = 'equity_parent_company_Q'
df_fin = process_finfactor_and_save_data(df_singleQ, col2, open)

BP = df_fin / mc
BP.to_pickle('D:/redata/factor/BP@factor.pkl')

In [168]:
# EP 因子

#单季度
col = 'net_profit'
df_single_Q = single_Q(dfaa,col)

col2 = 'net_profit_Q'
df_fin = process_finfactor_and_save_data(df_single_Q, col2, open)

EP = df_fin / mc
EP.to_pickle('D:/redata/factor/EP@factor.pkl')

In [169]:
# EPttm 因子

#单季度
col = 'net_profit'
df_single_Q = single_Q(dfaa,col)
df_merge = df_single_Q.merge(mls,on=['report_period'],how='left')

col2 = 'net_profit'
df_computed = compute_ttm(df_merge,col2)

col3 = 'net_profit_TTM'
df_fin = process_finfactor_and_save_data(df_computed, col3, open)
EP_ttm = df_fin / mc

EP_ttm.to_pickle('D:/redata/factor/EP_ttm@factor.pkl')

In [170]:
# CFP_ttm

#单季度
col = 'cash_flow_from_operating_activities'
df_single_Q = single_Q(dfbb,col)
df_merge = df_single_Q.merge(mlss,on=['report_period'],how='left')

# ttm12月累计——QoQ环比增速——yoy同比增速——Diff当季-上季数值
df_ttm = compute_ttm(df_merge,col)

col1 = 'cash_flow_from_operating_activities_TTM'
df_fin = process_finfactor_and_save_data(df_ttm, col1, open)

cfp = df_fin/mc
cfp.to_pickle('D:/redata/factor/cfp_ttm@factor.pkl')

In [171]:
# 归母ROE

#单季度_np
col1 = 'net_profit_parent_company'
df_single_Q_nppc = single_Q(dfaa,col1)

col2 = 'net_profit_parent_company_Q'
df_nppc = process_finfactor_and_save_data(df_single_Q_nppc, col2, open)

#单季度_epc
col3 = 'equity_parent_company'
df_single_Q_epc = single_Q(dfcc,col3)

col4 = 'equity_parent_company_Q'
df_epc = process_finfactor_and_save_data(df_single_Q_epc, col4, open)

#单季roe
roe = df_nppc/df_epc
roe.to_pickle('D:/redata/factor/roe@factor.pkl')

In [172]:
# 归母ROE同比

df_roe = df_single_Q_nppc.merge(df_single_Q_epc, on=['ticker', 'date', 'report_period'], how='left')
df_roe['roe_Q'] = df_roe['net_profit_parent_company_Q']/ df_roe['equity_parent_company_Q']
df_merge = df_roe.merge(mls,on=['report_period'],how='left')

# ttm12月累计——QoQ环比增速——yoy同比增速——Diff当季-上季数值
col5 = 'roe_Q'
df_computed = compute_yoy(df_merge,col5)
col6 = 'roe_Q_yoy'
delt_roe = process_finfactor_and_save_data(df_computed, col6, open)

delt_roe.to_pickle('D:/redata/factor/roe_yoy@factor.pkl')

In [173]:
# 归母ROA

#单季度_np
col1 = 'net_profit_parent_company'
df_single_Q_nppc = single_Q(dfaa,col1)

col2 = 'net_profit_parent_company_Q'
df_nppc = process_finfactor_and_save_data(df_single_Q_nppc, col2, open)

#单季度_epc
col3 = 'total_assets'
df_single_Q_ta = single_Q(dfcc,col3)

col4 = 'total_assets_Q'
df_ta = process_finfactor_and_save_data(df_single_Q_ta, col4, open)

#单季roe
roa = df_nppc/df_ta
roa.to_pickle('D:/redata/factor/roa@factor.pkl')

In [174]:
# 归母ROA同比

df_roa = df_single_Q_nppc.merge(df_single_Q_ta, on=['ticker', 'date', 'report_period'], how='left')
df_roa['roa_Q'] = df_roa['net_profit_parent_company_Q']/ df_roa['total_assets_Q']
df_merge = df_roa.merge(mls,on=['report_period'],how='left')

# ttm12月累计——QoQ环比增速——yoy同比增速——Diff当季-上季数值
col5 = 'roa_Q'
df_computed = compute_yoy(df_merge,col5)
col6 = 'roa_Q_yoy'
delt_roa = process_finfactor_and_save_data(df_computed, col6, open)

delt_roa.to_pickle('D:/redata/factor/roa_yoy@factor.pkl')

# 行业市值中性化

In [175]:
def compute_condition(new_stk=20):
    drop_new_stk = adj_open.rolling(window=new_stk, min_periods=new_stk).mean()
    drop_new_stk = ~np.isnan(drop_new_stk)
    df = st_status * halt_status * drop_new_stk
    df = df.replace(True, 1).replace(False, np.nan)
    return df.loc['2017':]

In [176]:
risk.tail(1)

,date,ticker,logcmv,retsum120,10,11,12,20,21,22,...,41,42,43,50,60,61,62,63,70,999
8143977,2024-12-27,689009,21.662996,0.284347,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [177]:
def process_factor(factor_path, output_path, final_universe, risk, risk_cols):
    df = pd.read_pickle(factor_path)
    df = df.tail(20)
    df = df.replace([np.inf, -np.inf], np.nan)
    df_clean = df * final_universe
    df_clean = df_clean.dropna(how='all', axis=0)
    
    factor_type = 'ind'
    factor_rank = factor_neutralize(df_clean, factor_type, if_rank=True, risk=risk, risk_cols=risk_cols)
    factor_rank.to_pickle(output_path)

final_universe = compute_condition(new_stk = 10) # Define your final_universe DataFrame
risk = risk  # Define your risk DataFrame
risk_cols = ['logcmv', 'retsum120']

# List of factors and their corresponding paths
factors = [
    # ('D:/redata/factor/hpl_r_std_6m@factor.pkl','D:/redata/Neutraliza/hpl_r_std_6m@nu.pkl'),
    # ('D:/redata/factor/bias_5@factor.pkl','D:/redata/Neutraliza/bias_5@nu.pkl'),
    # ('D:/redata/factor/net_profit_Q_yoy@factor.pkl','D:/redata/Neutraliza/net_profit_Q_yoy@nu.pkl'),
    # ('D:/redata/factor/qfa_ocftosales@factor.pkl','D:/redata/Neutraliza/qfa_ocftosales@nu.pkl'),
    # ('D:/redata/factor/mmt_avg_M@factor.pkl','D:/redata/Neutraliza/mmt_avg_M@nu.pkl'),

    # 组合中 #
    # ('D:/redata/factor/operating_revenue_Q_yoy@factor.pkl','D:/redata/Neutraliza/operating_revenue_Q_yoy@nu.pkl'),
    ('D:/redata/factor/roe_yoy@factor.pkl','D:/redata/Neutraliza/roe_yoy@nu.pkl'),
    ('D:/redata/factor/roa@factor.pkl','D:/redata/Neutraliza/roa@nu.pkl'),
    # ('D:/redata/factor/roe@factor.pkl','D:/redata/Neutraliza/roe@nu.pkl'),
    ('D:/redata/factor/roa_yoy@factor.pkl','D:/redata/Neutraliza/roa_yoy@nu.pkl'),
    
    ('D:/redata/factor/net_profit_parent_company_Q_yoy@factor.pkl','D:/redata/Neutraliza/net_profit_parent_company_Q_yoy@nu.pkl'),
    ('D:/redata/factor/cfp_ttm@factor.pkl','D:/redata/Neutraliza/cfp_ttm@nu.pkl'),
    ('D:/redata/factor/sp_ttm@factor.pkl','D:/redata/Neutraliza/sp_ttm@nu.pkl'),
    
    ('D:/redata/factor/EP_ttm@factor.pkl','D:/redata/Neutraliza/EP_ttm@nu.pkl'),
    # ('D:/redata/factor/EP@factor.pkl','D:/redata/Neutraliza/EP@nu.pkl'),
    # ('D:/redata/factor/BP@factor.pkl','D:/redata/Neutraliza/BP@nu.pkl'),
    
    ('D:/redata/factor/adjclose@factor.pkl','D:/redata/Neutraliza/adjclose@nu.pkl'),
    
    ('D:/redata/factor/maketcap@factor.pkl','D:/redata/Neutraliza/maketcap@nu.pkl'),
    
    ('D:/redata/factor/hml_r_std_6m@factor.pkl','D:/redata/Neutraliza/hml_r_std_6m@nu.pkl'),
    
    # ('D:/redata/factor/terra_20d@factor.pkl','D:/redata/Neutraliza/terra_20d@nu.pkl'),
    
    # ('D:/redata/factor/mmt_overnight_A@factor.pkl','D:/redata/Neutraliza/mmt_overnight_A@nu.pkl'),
    ('D:/redata/factor/mom_overnight_A@factor.pkl','D:/redata/Neutraliza/mom_overnight_A@nu.pkl'),
    
    # ('D:/redata/factor/mmt_off_limit_A@factor.pkl','D:/redata/Neutraliza/mmt_off_limit_A@nu.pkl'),
    # ('D:/redata/factor/mmt_normal_A@factor.pkl','D:/redata/Neutraliza/mmt_normal_A@nu.pkl'),
    ('D:/redata/factor/mom_A@factor.pkl','D:/redata/Neutraliza/mom_A@nu.pkl'),
    #&&&&&&&
    # ('D:/redata/factor/mmt_off_limit_M@factor.pkl','D:/redata/Neutraliza/mmt_off_limit_M@nu.pkl'),
    # ('D:/redata/factor/mmt_normal_M@factor.pkl','D:/redata/Neutraliza/mmt_normal_M@nu.pkl'),
    ('D:/redata/factor/mom_M@factor.pkl','D:/redata/Neutraliza/mom_M@nu.pkl'),
    
    # ('D:/redata/factor/mmt_intraday_M@factor.pkl', 'D:/redata/Neutraliza/mmt_intraday_M@nu.pkl'),
    # ('D:/redata/factor/mmt_range_M@factor.pkl','D:/redata/Neutraliza/mmt_range_M@nu.pkl'),
    ('D:/redata/factor/reverse_M@factor.pkl','D:/redata/Neutraliza/reverse_M@nu.pkl'),
    
    # ('D:/redata/factor/vol_up_std_6M@factor.pkl','D:/redata/Neutraliza/vol_up_std_6M@nu.pkl'),
    # ('D:/redata/factor/vol_highlow_std_6M@factor.pkl','D:/redata/Neutraliza/vol_highlow_std_6M@nu.pkl'),
    # ('D:/redata/factor/vol_upshadow_std_6M@factor.pkl','D:/redata/Neutraliza/vol_upshadow_std_6M@nu.pkl'),
    # ('D:/redata/factor/vol_w_downshadow_std_6M@factor.pkl','D:/redata/Neutraliza/vol_w_downshadow_std_6M@nu.pkl'),
    ('D:/redata/factor/vol_6M@factor.pkl','D:/redata/Neutraliza/vol_6M@nu.pkl'),
    
    # ('D:/redata/factor/liq_turn_std_6M@factor.pkl','D:/redata/Neutraliza/liq_turn_std_6M@nu.pkl'),
    ('D:/redata/factor/turnrate_6M@factor.pkl','D:/redata/Neutraliza/turnrate_6M@nu.pkl'),
    
    # ('D:/redata/factor/liq_vstd_1M@factor.pkl','D:/redata/Neutraliza/liq_vstd_1M@nu.pkl'),
    # ('D:/redata/factor/liq_amiihud_avg_1M@factor.pkl','D:/redata/Neutraliza/liq_amiihud_avg_1M@nu.pkl'),
    # ('D:/redata/factor/liq_shortcut_avg_1M@factor.pkl','D:/redata/Neutraliza/liq_shortcut_avg_1M@nu.pkl'),
    ('D:/redata/factor/ill_price_M@factor.pkl','D:/redata/Neutraliza/ill_price_M@nu.pkl'),
    
    # ('D:/redata/factor/corr_ret_turnd_1M@factor.pkl','D:/redata/Neutraliza/corr_ret_turnd_1M@nu.pkl'),
    # ('D:/redata/factor/corr_price_turn_1M@factor.pkl','D:/redata/Neutraliza/corr_price_turn_1M@nu.pkl'),
    # ('D:/redata/factor/corr_ret_turn_post_1M@factor.pkl','D:/redata/Neutraliza/corr_ret_turn_post_1M@nu.pkl'),
    ('D:/redata/factor/corrPV_M@factor.pkl','D:/redata/Neutraliza/corrPV_M@nu.pkl'),

    ('D:/redata/factor/volume_20d_fac@factor.pkl','D:/redata/Neutraliza/volume_20d_fac@nu.pkl'),
    # ('D:/redata/factor/volume_60d_ratio@factor.pkl','D:/redata/Neutraliza/volume_60d_ratio@nu.pkl'),
   
    # 组合中 #
    ]
    
# Process each factor
for factor_path, output_path in factors:
    process_factor(factor_path, output_path, final_universe, risk, risk_cols)

100%|██████████| 20/20 [00:00<00:00, 44.45it/s]


In [178]:
def compute_IC(factor_rank, ret_rank):
    IC = (factor_rank.corrwith(ret_rank, axis=1)).dropna()
    IC_mean = IC.mean()
    IC_std = IC.std()
    ICIR = IC_mean / IC_std
    ICres = pd.DataFrame([[IC_mean, ICIR]], columns=['ICmean', 'ICIR'])
    ICIR_120_weight = IC.rolling(120).apply(lambda x: x.mean() / x.std()).dropna()
    return IC, ICres, ICIR_120_weight

# Assuming factor_rank and ret_rank are defined elsewhere in your code
# factor_rank = ... # Define your factor_rank DataFrame
ret_rank = o1o2.rank(axis=1, ascending=True) #1日调仓
ret_rank = ret_rank.ffill()

factors_nu = [
    ('D:/redata/Neutraliza/bias_5@nu.pkl','D:/redata/ICIR/bias_5@ICIR.pkl'),
    ('D:/redata/Neutraliza/net_profit_Q_yoy@nu.pkl','D:/redata/ICIR/net_profit_Q_yoy@ICIR.pkl'),
    ('D:/redata/Neutraliza/operating_revenue_Q_yoy@nu.pkl','D:/redata/ICIR/operating_revenue_Q_yoy@ICIR.pkl'),
    ('D:/redata/Neutraliza/qfa_ocftosales@nu.pkl','D:/redata/ICIR/qfa_ocftosales@ICIR.pkl'),
    ('D:/redata/Neutraliza/BP@nu.pkl','D:/redata/ICIR/BP@ICIR.pkl'),
    ('D:/redata/Neutraliza/net_profit_parent_company_Q_yoy@nu.pkl','D:/redata/ICIR/net_profit_parent_company_Q_yoy@ICIR.pkl'),
    ('D:/redata/Neutraliza/roe_yoy@nu.pkl','D:/redata/ICIR/roe_yoy@ICIR.pkl'),
    ('D:/redata/Neutraliza/roa@nu.pkl','D:/redata/ICIR/roa@ICIR.pkl'),
    ('D:/redata/Neutraliza/terra_20d@nu.pkl','D:/redata/ICIR/terra_20d@ICIR.pkl'),
    ('D:/redata/Neutraliza/hml_r_std_6m@nu.pkl','D:/redata/ICIR/hml_r_std_6m@ICIR.pkl'),
    ('D:/redata/Neutraliza/EP_ttm@nu.pkl','D:/redata/ICIR/EP_ttm@ICIR.pkl'),
    ('D:/redata/Neutraliza/EP@nu.pkl','D:/redata/ICIR/EP@ICIR.pkl'),
    ('D:/redata/Neutraliza/maketcap@nu.pkl','D:/redata/ICIR/maketcap@ICIR.pkl'),
    ('D:/redata/Neutraliza/adjclose@nu.pkl','D:/redata/ICIR/adjclose@ICIR.pkl'),
    ('D:/redata/Neutraliza/sp_ttm@nu.pkl','D:/redata/ICIR/sp_ttm@ICIR.pkl'),
    ('D:/redata/Neutraliza/roe@nu.pkl','D:/redata/ICIR/roe@ICIR.pkl'),
    ('D:/redata/Neutraliza/roa_yoy@nu.pkl','D:/redata/ICIR/roa_yoy@ICIR.pkl'),
    ('D:/redata/Neutraliza/cfp_ttm@nu.pkl','D:/redata/ICIR/cfp_ttm@ICIR.pkl'),

    
    # ('D:/redata/Neutraliza/mmt_overnight_A@nu.pkl','D:/redata/ICIR/mmt_overnight_A@ICIR.pkl'),
    ('D:/redata/Neutraliza/mom_overnight_A@nu.pkl','D:/redata/ICIR/mom_overnight_A@ICIR.pkl'),
    
    # ('D:/redata/Neutraliza/mmt_normal_A@nu.pkl','D:/redata/ICIR/mmt_normal_A@ICIR.pkl'),
    # ('D:/redata/Neutraliza/mmt_off_limit_A@nu.pkl','D:/redata/ICIR/mmt_off_limit_A@ICIR.pkl'),
    ('D:/redata/Neutraliza/mom_A@nu.pkl','D:/redata/ICIR/mom_A@ICIR.pkl'),

    # ('D:/redata/Neutraliza/mmt_normal_M@nu.pkl','D:/redata/ICIR/mmt_normal_M@ICIR.pkl'),
    # ('D:/redata/Neutraliza/mmt_off_limit_M@nu.pkl','D:/redata/ICIR/mmt_off_limit_M@ICIR.pkl'),
    ('D:/redata/Neutraliza/mom_M@nu.pkl','D:/redata/ICIR/mom_M@ICIR.pkl'),
    
    # ('D:/redata/Neutraliza/mmt_intraday_M@nu.pkl', 'D:/redata/ICIR/mmt_intraday_M@ICIR.pkl'),
    # ('D:/redata/Neutraliza/mmt_range_M@nu.pkl','D:/redata/ICIR/mmt_range_M@ICIR.pkl'),
    ('D:/redata/Neutraliza/reverse_M@nu.pkl','D:/redata/ICIR/reverse_M@ICIR.pkl'),
    
    # ('D:/redata/Neutraliza/vol_up_std_6M@nu.pkl','D:/redata/ICIR/vol_up_std_6M@ICIR.pkl'),
    # ('D:/redata/Neutraliza/vol_highlow_std_6M@nu.pkl','D:/redata/ICIR/vol_highlow_std_6M@ICIR.pkl'),
    # ('D:/redata/Neutraliza/vol_upshadow_std_6M@nu.pkl','D:/redata/ICIR/vol_upshadow_std_6M@ICIR.pkl'),
    # ('D:/redata/Neutraliza/vol_w_downshadow_std_6M@nu.pkl','D:/redata/ICIR/vol_w_downshadow_std_6M@ICIR.pkl'),
    ('D:/redata/Neutraliza/vol_6M@nu.pkl','D:/redata/ICIR/vol_6M@ICIR.pkl'),
    
    # ('D:/redata/Neutraliza/liq_turn_std_6M@nu.pkl','D:/redata/ICIR/liq_turn_std_6M@ICIR.pkl'),
    ('D:/redata/Neutraliza/turnrate_6M@nu.pkl','D:/redata/ICIR/turnrate_6M@ICIR.pkl'),
    
    # ('D:/redata/Neutraliza/liq_vstd_1M@nu.pkl','D:/redata/ICIR/liq_vstd_1M@ICIR.pkl'),
    # ('D:/redata/Neutraliza/liq_amiihud_avg_1M@nu.pkl','D:/redata/ICIR/liq_amiihud_avg_1M@ICIR.pkl'),
    # ('D:/redata/Neutraliza/liq_shortcut_avg_1M@nu.pkl','D:/redata/ICIR/liq_shortcut_avg_1M@ICIR.pkl'),
    ('D:/redata/Neutraliza/ill_price_M@nu.pkl','D:/redata/ICIR/ill_price_M@ICIR.pkl'),
    
    # ('D:/redata/Neutraliza/corr_ret_turnd_1M@nu.pkl','D:/redata/ICIR/corr_ret_turnd_1M@ICIR.pkl'),
    # ('D:/redata/Neutraliza/corr_price_turn_1M@nu.pkl','D:/redata/ICIR/corr_price_turn_1M@ICIR.pkl'),
    # ('D:/redata/Neutraliza/corr_ret_turn_post_1M@nu.pkl','D:/redata/ICIR/corr_ret_turn_post_1M@ICIR.pkl'),
    ('D:/redata/Neutraliza/corrPV_M@nu.pkl','D:/redata/ICIR/corrPV_M@ICIR.pkl'),

    ('D:/redata/Neutraliza/volume_20d_fac@nu.pkl','D:/redata/ICIR/volume_20d_fac@ICIR.pkl'),
    ('D:/redata/Neutraliza/volume_60d_ratio@nu.pkl','D:/redata/ICIR/volume_60d_ratio@ICIR.pkl'),
]

# Process each factor
for factor_path, output_path in factors_nu:
    factor_rank = pd.read_pickle(factor_path)
# Assuming ret_rank is defined and available
    IC, ICres, ICIR_120_weight = compute_IC(factor_rank, ret_rank)
# Optionally save ICIR_120_weight if needed
    ICIR_120_weight.to_pickle(output_path.replace('.pkl', '_120_weight.pkl'))
    ICres.to_pickle(output_path)

# 相关性检验

# 选股

cor_df = pd.concat([i3,i5,i7,i14,i15,i18,i19,i21,i22,i23,i24,i25,i26,i28,i29], axis=1) # ,i21,i22,i23,i24,i25,i26,i28,i29
cor_df = cor_df.dropna(axis=0)

correlation_matrix = (cor_df.corr()).dropna()
sns.heatmap(correlation_matrix, annot=True,cmap='coolwarm',vmin=-1,vmax=1)
plt.title('Correlation Heatmap')
plt.show()

In [179]:
# # 有ICIR加权时，权重有延迟。会导致选股失败。
# f2 = pd.read_pickle('D:/redata/CombineFactors/roe@combine.pkl')
# f3 = pd.read_pickle('D:/redata/CombineFactors/roe_yoy@combine.pkl')
# f4 = pd.read_pickle('D:/redata/CombineFactors/roa@combine.pkl')
# f5 = pd.read_pickle('D:/redata/CombineFactors/hml_r_std_6m@combine.pkl')
# f6 = pd.read_pickle('D:/redata/CombineFactors/net_profit_Q_yoy@combine.pkl')
# f7 = pd.read_pickle('D:/redata/CombineFactors/net_profit_parent_company_Q_yoy@combine.pkl')
# f8 = pd.read_pickle('D:/redata/CombineFactors/operating_revenue_Q_yoy@combine.pkl')
# f9 = pd.read_pickle('D:/redata/CombineFactors/BP@combine.pkl')
# f10 = pd.read_pickle('D:/redata/CombineFactors/bias_5@combine.pkl')
# f11 = pd.read_pickle('D:/redata/CombineFactors/qfa_ocftosales@combine.pkl')
# f12 = pd.read_pickle('D:/redata/CombineFactors/terra_20d@combine.pkl')
# f14 = pd.read_pickle('D:/redata/CombineFactors/EP_ttm@combine.pkl')
# f15 = pd.read_pickle('D:/redata/CombineFactors/sp_ttm@combine.pkl')
# f16 = pd.read_pickle('D:/redata/CombineFactors/roa_yoy@combine.pkl')
# f17 = pd.read_pickle('D:/redata/CombineFactors/EP@combine.pkl')
# f18 = pd.read_pickle('D:/redata/CombineFactors/adjclose@combine.pkl')
# f19 = pd.read_pickle('D:/redata/CombineFactors/maketcap@combine.pkl')
# f20 = pd.read_pickle('D:/redata/CombineFactors/cfp_ttm@combine.pkl')

# # f21 = pd.read_pickle('D:/redata/CombineFactors/mmt_overnight_A@combine.pkl')
# f21 = pd.read_pickle('D:/redata/CombineFactors/mom_overnight_A@combine.pkl')

# # f22 = pd.read_pickle('D:/redata/CombineFactors/mmt_intraday_M@combine.pkl')
# # f23 = pd.read_pickle('D:/redata/CombineFactors/mmt_range_M@combine.pkl')  
# f22 = pd.read_pickle('D:/redata/CombineFactors/reverse_M@combine.pkl')

# # f24 = pd.read_pickle('D:/redata/CombineFactors/mmt_normal_A@combine.pkl')
# # f25 = pd.read_pickle('D:/redata/CombineFactors/mmt_off_limit_A@combine.pkl')
# f23 = pd.read_pickle('D:/redata/CombineFactors/mom_A@combine.pkl')

# # f26 = pd.read_pickle('D:/redata/CombineFactors/vol_up_std_6M@combine.pkl')
# # f27 = pd.read_pickle('D:/redata/CombineFactors/vol_highlow_std_6M@combine.pkl')
# # f28 = pd.read_pickle('D:/redata/CombineFactors/vol_upshadow_std_6M@combine.pkl')
# # f29 = pd.read_pickle('D:/redata/CombineFactors/vol_w_downshadow_std_6M@combine.pkl')
# f24 = pd.read_pickle('D:/redata/CombineFactors/vol_6M@combine.pkl')

# # f30 = pd.read_pickle('D:/redata/CombineFactors/liq_turn_std_6M@combine.pkl')
# f25 = pd.read_pickle('D:/redata/CombineFactors/turnrate_6M@combine.pkl')

# # f31 = pd.read_pickle('D:/redata/CombineFactors/liq_vstd_1M@combine.pkl')
# # f32 = pd.read_pickle('D:/redata/CombineFactors/liq_amiihud_avg_1M@combine.pkl')
# # f33 = pd.read_pickle('D:/redata/CombineFactors/liq_shortcut_avg_1M@combine.pkl')
# f26 = pd.read_pickle('D:/redata/CombineFactors/ill_price_M@combine.pkl')

# # f34 = pd.read_pickle('D:/redata/CombineFactors/corr_ret_turnd_1M@combine.pkl')
# # f35 = pd.read_pickle('D:/redata/CombineFactors/corr_price_turn_1M@combine.pkl')
# # f36 = pd.read_pickle('D:/redata/CombineFactors/corr_ret_turn_post_1M@combine.pkl')
# f27 = pd.read_pickle('D:/redata/CombineFactors/corrPV_M@combine.pkl')

# # f37 = pd.read_pickle('D:/redata/CombineFactors/mmt_normal_M@combine.pkl')
# # f38 = pd.read_pickle('D:/redata/CombineFactors/mmt_off_limit_M@combine.pkl')
# f28 = pd.read_pickle('D:/redata/CombineFactors/mom_M@combine.pkl')

# f39 = pd.read_pickle('D:/redata/Neutraliza/volume_20d_fac@nu.pkl')
# f40 = pd.read_pickle('D:/redata/Neutraliza/volume_60d_ratio@nu.pkl')

In [180]:
f2 = pd.read_pickle('D:/redata/Neutraliza/roe@nu.pkl')
f3 = pd.read_pickle('D:/redata/Neutraliza/roe_yoy@nu.pkl')
f4 = pd.read_pickle('D:/redata/Neutraliza/roa@nu.pkl')
f5 = pd.read_pickle('D:/redata/Neutraliza/hml_r_std_6m@nu.pkl')
f6 = pd.read_pickle('D:/redata/Neutraliza/net_profit_Q_yoy@nu.pkl')
f7 = pd.read_pickle('D:/redata/Neutraliza/net_profit_parent_company_Q_yoy@nu.pkl')
f8 = pd.read_pickle('D:/redata/Neutraliza/operating_revenue_Q_yoy@nu.pkl')
f9 = pd.read_pickle('D:/redata/Neutraliza/BP@nu.pkl')
f10 = pd.read_pickle('D:/redata/Neutraliza/bias_5@nu.pkl')
f11 = pd.read_pickle('D:/redata/Neutraliza/qfa_ocftosales@nu.pkl')
f12 = pd.read_pickle('D:/redata/Neutraliza/terra_20d@nu.pkl')
f14 = pd.read_pickle('D:/redata/Neutraliza/EP_ttm@nu.pkl')
f15 = pd.read_pickle('D:/redata/Neutraliza/sp_ttm@nu.pkl')
f16 = pd.read_pickle('D:/redata/Neutraliza/roa_yoy@nu.pkl')
f17 = pd.read_pickle('D:/redata/Neutraliza/EP@nu.pkl')
f18 = pd.read_pickle('D:/redata/Neutraliza/adjclose@nu.pkl')
f19 = pd.read_pickle('D:/redata/Neutraliza/maketcap@nu.pkl')
f20 = pd.read_pickle('D:/redata/Neutraliza/cfp_ttm@nu.pkl')

# f21 = pd.read_pickle('D:/redata/Neutraliza/mmt_overnight_A@nu.pkl')
fh21 = pd.read_pickle('D:/redata/Neutraliza/mom_overnight_A@nu.pkl')

# f22 = pd.read_pickle('D:/redata/Neutraliza/mmt_intraday_M@nu.pkl')
# f23 = pd.read_pickle('D:/redata/Neutraliza/mmt_range_M@nu.pkl')  
fh22 = pd.read_pickle('D:/redata/Neutraliza/reverse_M@nu.pkl')

# f24 = pd.read_pickle('D:/redata/Neutraliza/mmt_normal_A@nu.pkl')
# f25 = pd.read_pickle('D:/redata/Neutraliza/mmt_off_limit_A@nu.pkl')
fh23 = pd.read_pickle('D:/redata/Neutraliza/mom_A@nu.pkl')

# f26 = pd.read_pickle('D:/redata/Neutraliza/vol_up_std_6M@nu.pkl')
# f27 = pd.read_pickle('D:/redata/Neutraliza/vol_highlow_std_6M@nu.pkl')
# f28 = pd.read_pickle('D:/redata/Neutraliza/vol_upshadow_std_6M@nu.pkl')
# f29 = pd.read_pickle('D:/redata/Neutraliza/vol_w_downshadow_std_6M@nu.pkl')
fh24 = pd.read_pickle('D:/redata/Neutraliza/vol_6M@nu.pkl')

# f30 = pd.read_pickle('D:/redata/Neutraliza/liq_turn_std_6M@nu.pkl')
fh25 = pd.read_pickle('D:/redata/Neutraliza/turnrate_6M@nu.pkl')

# f31 = pd.read_pickle('D:/redata/Neutraliza/liq_vstd_1M@nu.pkl')
# f32 = pd.read_pickle('D:/redata/Neutraliza/liq_amiihud_avg_1M@nu.pkl')
# f33 = pd.read_pickle('D:/redata/Neutraliza/liq_shortcut_avg_1M@nu.pkl')
fh26 = pd.read_pickle('D:/redata/Neutraliza/ill_price_M@nu.pkl')

# f34 = pd.read_pickle('D:/redata/Neutraliza/corr_ret_turnd_1M@nu.pkl')
# f35 = pd.read_pickle('D:/redata/Neutraliza/corr_price_turn_1M@nu.pkl')
# f36 = pd.read_pickle('D:/redata/Neutraliza/corr_ret_turn_post_1M@nu.pkl')
fh27 = pd.read_pickle('D:/redata/Neutraliza/corrPV_M@nu.pkl')

# f37 = pd.read_pickle('D:/redata/Neutraliza/mmt_normal_M@nu.pkl')
# f38 = pd.read_pickle('D:/redata/Neutraliza/mmt_off_limit_M@nu.pkl')
fh28 = pd.read_pickle('D:/redata/Neutraliza/mom_M@nu.pkl')

f39 = pd.read_pickle('D:/redata/Neutraliza/volume_20d_fac@nu.pkl')
f40 = pd.read_pickle('D:/redata/Neutraliza/volume_60d_ratio@nu.pkl')

In [181]:
comb_mean = f3+f5+f7+f14+f15+f18+f19+fh21+fh22+fh23/2+fh24/2+fh25/2+fh26+fh28/2+f39
# comb_mean = f3+f5+f7+f14+f15+f18+f19+fh21+fh22+fh23+fh24+fh25+fh26+fh28+f39
# comb_mean = f5+f14+f18+f19+f20+fh21+fh22+fh23/2+fh24/2+fh25/2+fh26+fh28/2+f39

comb_mean.to_pickle('D:/redata/factor/comb_mean.pkl')

#市值分层read_pickle
comb_mean_s = comb_mean * small
comb_mean_s.to_pickle('D:/redata/factor/comb_mean_s.pkl')

comb_mean_m = comb_mean * median
comb_mean_m.to_pickle('D:/redata/factor/comb_mean_m.pkl')

comb_mean_sm = comb_mean * sm
comb_mean_sm.to_pickle('D:/redata/factor/comb_mean_sm.pkl')

comb_mean_l = comb_mean * large
comb_mean_l.to_pickle('D:/redata/factor/comb_mean_l.pkl')

# 5、10、20日
# a = position(comb_mean_sm, 5)
# a.to_pickle('D:/redata/factor/comb_mean_sm5d.pkl')
# b = position(comb_mean_sm, 10)
# b.to_pickle('D:/redata/factor/comb_mean_sm10d.pkl')
# d = position(comb_mean_sm, 20)
# d.to_pickle('D:/redata/factor/comb_mean_sm20d.pkl')

In [182]:
comb_mean.tail(2)

ticker,000001,000002,000004,000006,000007,000008,000009,000010,000011,000012,...,688787,688788,688789,688793,688798,688799,688800,688819,688981,689009
date,,,,,,,,,,,,,,,,,,,,,
2024-12-26,35046.5,28990.0,26073.0,NaN,35058.0,27422.5,33863.5,NaN,28180.5,32004.5,...,27892.5,36908.0,30163.0,22634.0,34218.5,33591.5,36506.0,40619.0,36869.0,34698.5
2024-12-27,35216.5,29156.0,26113.0,29256.0,33967.5,27124.5,34443.0,28308.0,29017.0,31728.0,...,27511.5,37529.0,30345.5,22879.5,34767.5,33095.5,36174.0,40987.0,36576.5,34130.0


In [183]:
df = pd.read_pickle('D:/redata/factor/comb_mean_sm.pkl')###读入因子值
df = df.replace(np.inf,np.nan).replace(-np.inf,np.nan)
#剔除 新股 st halt
df_clean = (df * final_universe)
df_clean = df_clean.dropna(how='all',axis=0)
factor_type = 'hist'  # 'hist' 因子原值， 'ind' 对因子做行业中性
factor_rank = factor_neutralize(df_clean,factor_type,if_rank=True,risk=risk,risk_cols=['logcmv','retsum120']) 

select_num_stocks(n=10)

ticker,300552,301071,301141,301230,301302,301328,301368,301388,301508,301516
date,,,,,,,,,,
2024-12-27,1,1,1,1,1,1,1,1,1,1


### 通达信选股